In [18]:
import json # library to handle JSON files

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

import pandas as pd
import numpy as np

import time

from bs4 import BeautifulSoup

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## 行政区划

### 获取北京市行政区划

In [19]:
# 获取北京市行政区划
#https://en.wikipedia.org/wiki/Beijing#Politics_and_government
#https://zh.wikipedia.org/wiki/%E5%8C%97%E4%BA%AC%E5%B8%82#%E7%B6%93%E6%BF%9F%E6%95%B8%E6%93%9A

In [20]:
df_divisions_bj = pd.read_csv("data/Administrative_divisions_BeiJing.csv")

In [21]:
df_divisions_bj.head()

,Division_code,Division,Area_in_km2,Total_population_2010,Urban_area_Population_2010,Seat,Postal_code,Subdistricts,Towns,Townships,Residential_communities,Villages,区划名称,政府驻地,划归北京时间,人口,面积,人口密度
0,110101,Dongcheng,41.82,919253,NaN,Jingshan Subdistrict,100000,17,,,216,,东城区,景山街道,共和国前,919000,42.0,21881
1,110102,Xicheng,50.33,1243315,NaN,Jinrong Street Subdistrict,100000,15,,,259,,西城区,金融街街道,共和国前,1243000,51.0,24372
2,110105,Chaoyang,454.78,3545137,3532257.0,Chaowai Subdistrict,100000,24,,19,358,5,朝阳区,朝外街道,共和国前,3545000,470.8,7530
3,110106,Fengtai,305.53,2112162,2098632.0,Fengtai Subdistrict,100000,16,2,3,254,73,丰台区,丰台街道,共和国前,2112000,304.2,6943
4,110107,Shijingshan,84.38,616083,NaN,Lugu Subdistrict,100000,9,,,130,,石景山区,鲁谷街道,共和国前,616000,89.8,6860


In [48]:
Divisions=df_divisions_bj['区划名称'].values

In [49]:
Divisions

array(['东城区', '西城区', '朝阳区', '丰台区', '石景山区', '海淀区', '门头沟区', '房山区', '通州区',
       '顺义区', '昌平区', '大兴区', '怀柔区', '平谷区', '密云区', '延庆区'], dtype=object)

### 全国行政区划

In [7]:
df_divisions = pd.read_csv("data/china_divsion_code.csv")

In [8]:
df_divisions['行政区划代码']=df_divisions['行政区划代码'].apply(str)
df_divisions['省直辖市']=df_divisions['行政区划代码'].apply(lambda x:x[:2])
df_divisions['省直辖市标志']=df_divisions['行政区划代码'].apply(lambda x:x[2:])
df_divisions['区县']=df_divisions['行政区划代码'].apply(lambda x:x[:4])
df_divisions['区县标志']=df_divisions['行政区划代码'].apply(lambda x:x[4:])
df_divisions.head()

,行政区划代码,单位名称,省直辖市,省直辖市标志,区县,区县标志
0,110000,北京市,11,0000,1100,00
1,110101,东城区,11,0101,1101,01
2,110102,西城区,11,0102,1101,02
3,110105,朝阳区,11,0105,1101,05
4,110106,丰台区,11,0106,1101,06


In [9]:
df_divisions_bsgs=df_divisions[df_divisions['区县'].isin(['1101','3101','4401','4403'])]
df_divisions_bsgs=df_divisions_bsgs[df_divisions_bsgs['区县标志']!='00']
df_divisions_bsgs.head()

# 北上广深  '1101','3101','4401','4403'

,行政区划代码,单位名称,省直辖市,省直辖市标志,区县,区县标志
1,110101,东城区,11,0101,1101,01
2,110102,西城区,11,0102,1101,02
3,110105,朝阳区,11,0105,1101,05
4,110106,丰台区,11,0106,1101,06
5,110107,石景山区,11,0107,1101,07


In [10]:
Divisions=df_divisions_bsgs['单位名称'].unique()
Divisions

array(['东城区', '西城区', '朝阳区', '丰台区', '石景山区', '海淀区', '门头沟区', '房山区', '通州区',
       '顺义区', '昌平区', '大兴区', '怀柔区', '平谷区', '密云区', '延庆区', '黄浦区', '徐汇区',
       '长宁区', '静安区', '普陀区', '虹口区', '杨浦区', '闵行区', '宝山区', '嘉定区', '浦东新区',
       '金山区', '松江区', '青浦区', '奉贤区', '崇明区', '荔湾区', '越秀区', '海珠区', '天河区',
       '白云区', '黄埔区', '番禺区', '花都区', '南沙区', '从化区', '增城区', '罗湖区', '福田区',
       '南山区', '宝安区', '龙岗区', '盐田区', '龙华区', '坪山区', '光明区'], dtype=object)

## 获取国内金融机构

In [11]:
from tabula import read_pdf 

#获取银行简称
def get_short_name(full_name):   
    
    try:
        i = full_name.index("行")
        short_name = full_name[:i+1]
    except:
        short_name = full_name
    
    return short_name

bank_df = read_pdf("data/银行业金融机构法人名单.pdf",pages='all')
bank_df['简称']=bank_df['中文全称'].apply(get_short_name)

In [12]:
bank_df.head(15)

,序号,中文全称,英文全称,金融许可证机构编码,机构类型,简称
0,1,国家开发银行,China Development Bank,G0001H111000001,开发性金融机构,国家开发银行
1,2,中国进出口银行,The Export-Import Bank of China,A0003H111000001,政策性银行,中国进出口银行
2,3,中国农业发展银行,Agricultural Development Bank of China,A0002H111000001,政策性银行,中国农业发展银行
3,4,中国工商银行股份有限公司,INDUSTRIAL AND COMMERCIAL BANK OF CHINA\rLIMITED,B0001H111000001,国有大型商业银行,中国工商银行
4,5,中国农业银行股份有限公司,AGRICULTURAL BANK OF CHINA LIMITED,B0002H111000001,国有大型商业银行,中国农业银行
5,6,中国银行股份有限公司,BANK OF CHINA LIMITED,B0003H111000001,国有大型商业银行,中国银行
6,7,中国建设银行股份有限公司,CHINA CONSTRUCTION BANK CORPORATION,B0004H111000001,国有大型商业银行,中国建设银行
7,8,交通银行股份有限公司,"BANK OF COMMUNICATIONS CO.,LTD.",B0005H131000001,国有大型商业银行,交通银行
8,9,中国邮政储蓄银行股份有限公司,"POSTAL SAVINGS BANK OF CHINA Co.,Ltd.",B0018H111000001,国有大型商业银行,中国邮政储蓄银行
9,10,中信银行股份有限公司,CHINA CITIC BANK CORPORATION LIMITED,B0006H111000001,股份制商业银行,中信银行


In [17]:
# 数据持久化
bank_df.to_csv("data/China_Financial_Institution_Directory.csv",index=False)


## 获取每个区县的银行坐标

### 参数配置

In [51]:
keyword="农业银行"
bank_names_1=['工商银行','农业银行','中国银行','建设银行','交通银行','邮政储蓄银行']
tag="金融"
#region="海淀区"
ak="10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ"  #9543
#ak="OW1n5FOFah6KiNd9RNKpHAd9BY8PR7Cz"   #1285
page_size='20'
page_num=0

In [52]:
def call_api(url):
    while True:
        try:
            results = requests.get(url).json()
            if results['status']==0:
                break
        except:
            print("调用接口失败")
            time.sleep(3)
            print("调用接口失败..",results['message'])
            time.sleep(3)
    return results

In [53]:
# 获取结果条数
def get_venues_num(keyword,tag,region,page_size,page_num,ak):
    url = 'http://api.map.baidu.com/place/v2/search?\
            query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
               keyword,
               tag,
               region,
               page_size,
               page_num,
               ak)
    
    results=call_api(url)
    venues_num=results['total']
    
    return venues_num   

In [54]:
# 获取查询结果
def get_venues(keyword,tag,region,page_size,page_num,ak):
    
    #先计算结果总条数
    venues_num = get_venues_num(keyword,tag,region,page_size,page_num,ak)
    #用总条数计算结果页数
    page_num_all=venues_num//20 + 1
    print("{} 区域内,{}-{}个数：{},总页数{}".format(region,keyword,tag,venues_num,page_num_all))
    
    
    results = pd.DataFrame()
    for num in range(page_num_all):
        #print("完成第{}页".format(num))
        url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
           keyword,
           tag,
           region,
           page_size,
           num,
           ak)
        result = call_api(url)
        #time.sleep(1)
        venues=result['results']
        nearby_venues = json_normalize(venues) # flatten JSON
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        results=pd.concat([results,nearby_venues],ignore_index=True)
        #print(results.shape)
        
    return results


# 获取查询结果
def get_venues_wgs(keyword,tag,region,page_size,page_num,ak):
    
    #先计算结果总条数
    venues_num = get_venues_num(keyword,tag,region,page_size,page_num,ak)
    #用总条数计算结果页数
    page_num_all=venues_num//20 + 1
    print("{} 区域内,{}-{}个数：{},总页数{}".format(region,keyword,tag,venues_num,page_num_all))
    
    
    results = pd.DataFrame()
    for num in range(page_num_all):
        #print("完成第{}页".format(num))
        url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&coord_type=1&page_size={}&page_num={}&output=json&ak={}'.format(
           keyword,
           tag,
           region,
           page_size,
           num,
           ak)
        result = call_api(url)
        #time.sleep(1)
        venues=result['results']
        nearby_venues = json_normalize(venues) # flatten JSON
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        results=pd.concat([results,nearby_venues],ignore_index=True)
        #print(results.shape)
        
    return results

### 多区县多银行

#### 遍历北上广深六大行

In [17]:
# 遍历北上广深所有区县
results = pd.DataFrame()
div_num=len(Divisions)

bank_names = bank_names_1

for i in range(div_num):
    region = Divisions[i]
    print("开始处理{}-{}/{}".format(i+1,region,div_num))
    
    
    # 获取单个区的银行坐标
    for keyword in bank_names:      
    
        result = get_venues(keyword,tag,region,page_size,page_num,ak)
        #time.sleep(1)
        #venues=result['results']
        #nearby_venues = json_normalize(venues) # flatten JSON

        #处理列名
        result.columns = [col.split(".")[-1] for col in result.columns]
        
        result['机构名称']=keyword
        #行并结果
        results=pd.concat([results,result],ignore_index=True,sort=False)
        print("处理{}-{}/{}完成，{}-{} 数据集维度：{}".format(i+1,region,div_num,keyword,tag,results.shape))   
        
        #results['机构名称']=keyword

        #各区数据持久化
        df_name="data/{}_bj_{}_{}_divisions.csv".format(region,keyword,tag)
        result.to_csv(df_name,index=False)

#汇总数据持久化    
df_name="data/{}_{}_{}_divisions.csv".format("北上广深","_".join(bank_names),tag)
results.to_csv(df_name,index=False)
print("{}-{}-{} 数据集维度：{}".format("北上广深","_".join(bank_names),tag,results.shape)) 

开始处理1-东城区/52
东城区 区域内,工商银行-金融个数：120,总页数7


/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:26: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



处理1-东城区/52完成，工商银行-金融 数据集维度：(120, 12)
东城区 区域内,农业银行-金融个数：30,总页数2
处理1-东城区/52完成，农业银行-金融 数据集维度：(150, 12)
东城区 区域内,中国银行-金融个数：42,总页数3
处理1-东城区/52完成，中国银行-金融 数据集维度：(192, 12)
东城区 区域内,建设银行-金融个数：51,总页数3
处理1-东城区/52完成，建设银行-金融 数据集维度：(243, 12)
东城区 区域内,交通银行-金融个数：16,总页数1
处理1-东城区/52完成，交通银行-金融 数据集维度：(259, 12)
东城区 区域内,邮政储蓄银行-金融个数：30,总页数2
处理1-东城区/52完成，邮政储蓄银行-金融 数据集维度：(289, 12)
开始处理2-西城区/52
西城区 区域内,工商银行-金融个数：199,总页数10
处理2-西城区/52完成，工商银行-金融 数据集维度：(488, 12)
西城区 区域内,农业银行-金融个数：43,总页数3
处理2-西城区/52完成，农业银行-金融 数据集维度：(531, 12)
西城区 区域内,中国银行-金融个数：67,总页数4
处理2-西城区/52完成，中国银行-金融 数据集维度：(598, 12)
西城区 区域内,建设银行-金融个数：79,总页数4
处理2-西城区/52完成，建设银行-金融 数据集维度：(677, 12)
西城区 区域内,交通银行-金融个数：28,总页数2
处理2-西城区/52完成，交通银行-金融 数据集维度：(705, 12)
西城区 区域内,邮政储蓄银行-金融个数：62,总页数4
处理2-西城区/52完成，邮政储蓄银行-金融 数据集维度：(767, 12)
开始处理3-朝阳区/52
朝阳区 区域内,工商银行-金融个数：248,总页数13
处理3-朝阳区/52完成，工商银行-金融 数据集维度：(1015, 12)
朝阳区 区域内,农业银行-金融个数：112,总页数6
处理3-朝阳区/52完成，农业银行-金融 数据集维度：(1127, 12)
朝阳区 区域内,中国银行-金融个数：180,总页数10
处理3-朝阳区/52完成，中国银行-金融 数据集维度：(1307, 12)
朝阳区 区域内,建设银行-金融个数：173,总页数9
处理3-朝阳区/52

处理21-普陀区/52完成，中国银行-金融 数据集维度：(5743, 12)
普陀区 区域内,建设银行-金融个数：29,总页数2
处理21-普陀区/52完成，建设银行-金融 数据集维度：(5772, 12)
普陀区 区域内,交通银行-金融个数：15,总页数1
处理21-普陀区/52完成，交通银行-金融 数据集维度：(5787, 12)
普陀区 区域内,邮政储蓄银行-金融个数：27,总页数2
处理21-普陀区/52完成，邮政储蓄银行-金融 数据集维度：(5814, 12)
开始处理22-虹口区/52
虹口区 区域内,工商银行-金融个数：68,总页数4
处理22-虹口区/52完成，工商银行-金融 数据集维度：(5882, 12)
虹口区 区域内,农业银行-金融个数：38,总页数2
处理22-虹口区/52完成，农业银行-金融 数据集维度：(5920, 12)
虹口区 区域内,中国银行-金融个数：33,总页数2
处理22-虹口区/52完成，中国银行-金融 数据集维度：(5953, 12)
虹口区 区域内,建设银行-金融个数：32,总页数2
处理22-虹口区/52完成，建设银行-金融 数据集维度：(5985, 12)
虹口区 区域内,交通银行-金融个数：11,总页数1
处理22-虹口区/52完成，交通银行-金融 数据集维度：(5996, 12)
虹口区 区域内,邮政储蓄银行-金融个数：33,总页数2
处理22-虹口区/52完成，邮政储蓄银行-金融 数据集维度：(6029, 12)
开始处理23-杨浦区/52
杨浦区 区域内,工商银行-金融个数：71,总页数4
处理23-杨浦区/52完成，工商银行-金融 数据集维度：(6100, 12)
杨浦区 区域内,农业银行-金融个数：35,总页数2
处理23-杨浦区/52完成，农业银行-金融 数据集维度：(6135, 12)
杨浦区 区域内,中国银行-金融个数：33,总页数2
处理23-杨浦区/52完成，中国银行-金融 数据集维度：(6168, 12)
杨浦区 区域内,建设银行-金融个数：38,总页数2
处理23-杨浦区/52完成，建设银行-金融 数据集维度：(6206, 12)
杨浦区 区域内,交通银行-金融个数：20,总页数2
处理23-杨浦区/52完成，交通银行-金融 数据集维度：(6226, 12)
杨浦区 区域内,邮政储蓄银行-

处理41-南沙区/52完成，中国银行-金融 数据集维度：(11399, 12)
南沙区 区域内,建设银行-金融个数：20,总页数2
处理41-南沙区/52完成，建设银行-金融 数据集维度：(11419, 12)
南沙区 区域内,交通银行-金融个数：0,总页数1
处理41-南沙区/52完成，交通银行-金融 数据集维度：(11419, 12)
南沙区 区域内,邮政储蓄银行-金融个数：15,总页数1
处理41-南沙区/52完成，邮政储蓄银行-金融 数据集维度：(11434, 12)
开始处理42-从化区/52
从化区 区域内,工商银行-金融个数：23,总页数2
处理42-从化区/52完成，工商银行-金融 数据集维度：(11457, 12)
从化区 区域内,农业银行-金融个数：25,总页数2
处理42-从化区/52完成，农业银行-金融 数据集维度：(11482, 12)
从化区 区域内,中国银行-金融个数：9,总页数1
处理42-从化区/52完成，中国银行-金融 数据集维度：(11491, 12)
从化区 区域内,建设银行-金融个数：25,总页数2
处理42-从化区/52完成，建设银行-金融 数据集维度：(11516, 12)
从化区 区域内,交通银行-金融个数：0,总页数1
处理42-从化区/52完成，交通银行-金融 数据集维度：(11516, 12)
从化区 区域内,邮政储蓄银行-金融个数：13,总页数1
处理42-从化区/52完成，邮政储蓄银行-金融 数据集维度：(11529, 12)
开始处理43-增城区/52
增城区 区域内,工商银行-金融个数：49,总页数3
处理43-增城区/52完成，工商银行-金融 数据集维度：(11578, 12)
增城区 区域内,农业银行-金融个数：35,总页数2
处理43-增城区/52完成，农业银行-金融 数据集维度：(11613, 12)
增城区 区域内,中国银行-金融个数：24,总页数2
处理43-增城区/52完成，中国银行-金融 数据集维度：(11637, 12)
增城区 区域内,建设银行-金融个数：43,总页数3
处理43-增城区/52完成，建设银行-金融 数据集维度：(11680, 12)
增城区 区域内,交通银行-金融个数：2,总页数1
处理43-增城区/52完成，交通银行-金融 数据集维度：(11682, 12)
增城区 

#### 遍历北京市所有区

In [55]:
# 遍历北京市所有区
results = pd.DataFrame()
div_num=len(Divisions)

for i in range(div_num):
    region = Divisions[i]
    print("开始处理{}-{}/{}".format(i+1,region,div_num))
    
    
    # 获取单个区的银行坐标
    
    result = get_venues_wgs(keyword,tag,region,page_size,page_num,ak)
    #time.sleep(1)
    #venues=result['results']
    #nearby_venues = json_normalize(venues) # flatten JSON
    
    #处理列名
    result.columns = [col.split(".")[-1] for col in result.columns]
    #行并结果
    results=pd.concat([results,result],ignore_index=True)
    print("处理{}-{}/{}完成，{}-{} 数据集维度：{}".format(i+1,region,div_num,keyword,tag,results.shape))   
    
    #各区数据持久化
    df_name="data/{}_bj_{}_{}_divisions.csv".format(region,keyword,tag)
    result.to_csv(df_name,index=False)

#汇总数据持久化    
df_name="data/{}_bj_{}_{}_divisions_wgs.csv".format("北京市",keyword,tag)
results.to_csv(df_name,index=False)
print("{}-{}-{} 数据集维度：{}".format("北京市",keyword,tag,results.shape)) 

开始处理1-东城区/16
东城区 区域内,农业银行-金融个数：30,总页数2


/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:57: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



处理1-东城区/16完成，农业银行-金融 数据集维度：(30, 11)
开始处理2-西城区/16
西城区 区域内,农业银行-金融个数：43,总页数3
处理2-西城区/16完成，农业银行-金融 数据集维度：(73, 11)
开始处理3-朝阳区/16
朝阳区 区域内,农业银行-金融个数：112,总页数6
处理3-朝阳区/16完成，农业银行-金融 数据集维度：(185, 11)
开始处理4-丰台区/16
丰台区 区域内,农业银行-金融个数：50,总页数3
处理4-丰台区/16完成，农业银行-金融 数据集维度：(235, 11)
开始处理5-石景山区/16
石景山区 区域内,农业银行-金融个数：14,总页数1
处理5-石景山区/16完成，农业银行-金融 数据集维度：(249, 11)
开始处理6-海淀区/16
海淀区 区域内,农业银行-金融个数：111,总页数6
处理6-海淀区/16完成，农业银行-金融 数据集维度：(360, 11)
开始处理7-门头沟区/16
门头沟区 区域内,农业银行-金融个数：6,总页数1
处理7-门头沟区/16完成，农业银行-金融 数据集维度：(366, 11)
开始处理8-房山区/16
房山区 区域内,农业银行-金融个数：34,总页数2
处理8-房山区/16完成，农业银行-金融 数据集维度：(400, 11)
开始处理9-通州区/16
通州区 区域内,农业银行-金融个数：34,总页数2
处理9-通州区/16完成，农业银行-金融 数据集维度：(434, 11)
开始处理10-顺义区/16
顺义区 区域内,农业银行-金融个数：38,总页数2
处理10-顺义区/16完成，农业银行-金融 数据集维度：(472, 11)
开始处理11-昌平区/16
昌平区 区域内,农业银行-金融个数：45,总页数3
处理11-昌平区/16完成，农业银行-金融 数据集维度：(517, 11)
开始处理12-大兴区/16
大兴区 区域内,农业银行-金融个数：51,总页数3
处理12-大兴区/16完成，农业银行-金融 数据集维度：(568, 11)
开始处理13-怀柔区/16
怀柔区 区域内,农业银行-金融个数：21,总页数2
处理13-怀柔区/16完成，农业银行-金融 数据集维度：(589, 11)
开始处理14-平谷区/16
平谷区 区域内,农业银行-金融个数：19,总页数

In [56]:
results

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市东城区建国门内大街69号,东城区,北京市,1,39.915342,116.426216,中国农业银行(总行),北京市,8599d63558a5a4392dfaa703,95599,8599d63558a5a4392dfaa703
1,朝阳门北大街13号,东城区,北京市,1,39.935785,116.439222,中国农业银行(北京市分行),北京市,d45be3778d313efc2d473303,(010)61127068,d45be3778d313efc2d473303
2,北京市东城区朝阳门内大街8号一层112号,东城区,北京市,1,39.930075,116.436869,中国农业银行(北京朝阳门支行),北京市,3996f7d6f0678c6476e5b6a8,(010)57929167,3996f7d6f0678c6476e5b6a8
3,北京市东城区安定门东大街28号雍和大厦东座一二层,东城区,北京市,1,39.955098,116.427525,中国农业银行(雍和支行),北京市,09185c564033885df11937ff,"(010)84046520,(010)84046458",09185c564033885df11937ff
4,北京市东城区安定门外大街190号(海南大厦一层),东城区,北京市,1,39.960631,116.414986,中国农业银行(青年湖支行),北京市,5c56edd6e004c85aa1d230ff,(010)64262519,5c56edd6e004c85aa1d230ff
5,北京市东城区东四北大街329号,东城区,北京市,1,39.935316,116.423512,中国农业银行(北京东四北支行),北京市,c21db9e5775d568300ff2dff,(010)64015010,c21db9e5775d568300ff2dff
6,北京市东城区崇文门外大街18号国瑞购物中心F1,东城区,北京市,1,39.905316,116.425879,中国农业银行(北京崇文门支行),北京市,242403a5b9b2041ded74f82c,(010)67183306,242403a5b9b2041ded74f82c
7,北京市东城区交道口东大街10号楼,东城区,北京市,1,39.946510,116.415657,中国农业银行(北京交道口支行),北京市,edd64ce1a70398913fe231ff,(010)64074868,edd64ce1a70398913fe231ff
8,北京市东城区和平里东街6区8号,东城区,北京市,1,39.962335,116.431311,中国农业银行(北京和平里东街支行),北京市,4ce10ce6f6c806a1441a32ff,(010)84224823,4ce10ce6f6c806a1441a32ff
9,北京市朝阳门北大街13号,东城区,北京市,1,39.935728,116.439737,中国农业银行24小时自助银行(北京市分行营业部),北京市,40b24ff63ae969b4d518b7b5,NaN,40b24ff63ae969b4d518b7b5


## 查询银行周边

### 银行数据分离

In [22]:
# 加载银行地理位置数据
df_six_big_bank_bsgs = pd.read_csv("data/北上广深_工商银行_农业银行_中国银行_建设银行_交通银行_邮政储蓄银行_金融_divisions.csv")

In [23]:
df_six_big_bank_bsgs.head()

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid,机构名称
0,北京市东城区崇文门外大街90号,东城区,北京市,1.0,39.901213,116.425594,中国工商银行(崇文门外大街支行),北京市,fc7675244b9b7b874e776ecb,(010)67114823,fc7675244b9b7b874e776ecb,工商银行
1,北京市东城区张自忠路10号华人一品大厦1层,东城区,北京市,1.0,39.939357,116.418112,中国工商银行(北京平安支行),北京市,cc788fde61b6c18888fc0665,(010)64078708,cc788fde61b6c18888fc0665,工商银行
2,东安门大街甲64号附近,东城区,北京市,1.0,39.921372,116.414488,中国工商银行,北京市,0a633a446fc210f417c8f692,95588,0a633a446fc210f417c8f692,工商银行
3,北京市东城区建国门内大街乙18号院1号楼1层,东城区,北京市,1.0,39.913227,116.430288,中国工商银行(北京中电财支行),北京市,b869b18dc8552c0ed42ce246,"(010)65218062,(010)65218057",b869b18dc8552c0ed42ce246,工商银行
4,北京市东城区柳罐胡同甲2号,东城区,北京市,1.0,39.911695,116.440193,中国工商银行(北京站支行),北京市,cd9d05ddfb88feb685420ec9,"(010)65238692,(010)65263920,(010)65263921",cd9d05ddfb88feb685420ec9,工商银行


In [24]:
# 北上广深农行
df_abc_bsgs=df_six_big_bank_bsgs[df_six_big_bank_bsgs['机构名称']=='农业银行']
print(df_abc_bsgs.shape)

#剔除非农行机构
df_abc_bsgs=df_abc_bsgs[df_abc_bsgs['name'].str.contains('银行|农业')]
print(df_abc_bsgs.shape)

# 筛选农行北上广深 ATM
df_abc_ATM_bsgs = df_abc_bsgs[df_abc_bsgs['name'].str.contains('ATM|自助')]
print(df_abc_bsgs.shape)

# 筛选农行北京 ATM
df_abc_ATM_bj=df_abc_ATM_bsgs[df_abc_ATM_bsgs['city']=='北京市']
df_abc_ATM_bj=df_abc_ATM_bj.reset_index(drop=True)
print(df_abc_ATM_bj.shape)

# 筛选农行上海 ATM
df_abc_ATM_sh=df_abc_ATM_bsgs[df_abc_ATM_bsgs['city']=='上海市']
print(df_abc_ATM_sh.shape)

# 筛选农行广州 ATM
df_abc_ATM_gz=df_abc_ATM_bsgs[df_abc_ATM_bsgs['city']=='广州市']
print(df_abc_ATM_gz.shape)

# 筛选农行深圳 ATM
df_abc_ATM_sz=df_abc_ATM_bsgs[df_abc_ATM_bsgs['city']=='深圳市']
print(df_abc_ATM_sz.shape)

(3036, 12)
(2989, 12)
(2989, 12)
(396, 12)
(481, 12)
(375, 12)
(669, 12)


In [28]:
## 坐标系转换  百度坐标转WGS
from coordTransform_utils import bd09_to_wgs84 as b2g
import tqdm

In [38]:
#df_abc_ATM_bj[['lat','lng']].apply(b2g)
lat_wgs =[]
lng_wgs=[]
for index,row in df_abc_ATM_bj.iterrows():
    
    lat,lng = b2g(row['lng'],row['lat'])
    
    lat_wgs.append(lat)
    lng_wgs.append(lng)
    print(index,row['name'],lat,lng)
    

0 中国农业银行24小时自助银行(北京市分行营业部) 116.42701811374916 39.928409460271745
1 中国农业银行ATM(和平里东街) 116.42056480417568 39.954462763116894
2 中国农业银行ATM(国瑞购物中心) 116.41414950026507 39.897035547888876
3 中国农业银行ATM(建国门内大街) 116.4133907837023 39.90775181438315
4 中国农业银行24小时自助银行(北京交道口支行) 116.4030622497937 39.93884256645369
5 中国农业银行24小时自助银行(国瑞购物中心) 116.41305487732609 39.897713610612215
6 中国农业银行24小时自助银行(东单支行) 116.41132467518256 39.90936145892309
7 中国农业银行24小时自助银行服务(北京富贵园分理处) 116.42725716486004 39.89321065736577
8 中国农业银行24小时自助银行服务(崇文支行营业部) 116.4108614233326 39.8923148863504
9 中国农业银行24小时自助银行(北京和平里东街支行) 116.41867928175266 39.95488183596022
10 中国农业银行24小时自助银行(北京雍和支行) 116.41495419446862 39.947536075677924
11 中国农业银行24小时自助银行(北京青年湖支行) 116.40244316646333 39.9528271565366
12 中国农业银行24小时自助银行(北京东城支行营业室) 116.41892444373755 39.913870256004806
13 中国农业银行24小时自助银行服务(北京朝阳门支行) 116.42405541668644 39.92266239423324
14 中国农业银行24小时自助银行(北京东四北支行) 116.41096049771738 39.927741061735546
15 中国农业银行24小时自助银行服务(北京工体路支行) 116.43255917543925 39.931677844

364 中国农业银行24小时自助银行(北京紫贵支行) 117.09349185726626 40.14312681858357
365 北京农商银行24小时自助银行服务 117.1158763628185 40.15330725536689
366 中国农业银行ATM(东源路) 116.84650114362842 40.38014656445944
367 中国农业银行ATM(司水路) 117.26286430971628 40.64452343277029
368 中国农业银行24小时自助银行 116.85203405751277 40.38497074384638
369 中国农业银行ATM(超市发檀西店) 116.86053775765069 40.380110331432576
370 中国农业银行24小时自助银行(东大街分理处) 116.84675302533314 40.375357297833624
371 中国农业银行24小时自助银行(新中街) 116.84531604966475 40.38160810794792
372 中国农业银行ATM(鼓楼南大街物美超市) 116.83953563819696 40.373989514883796
373 中国农业银行24小时自助银行(西统路) 116.7491381431627 40.364770653546934
374 中国农业银行ATM(经济开发区水源路) 116.79900957610432 40.35594261876074
375 中国农业银行24小时自助银行(新北路) 116.85704301737248 40.387857345771685
376 中国农业银行24小时自助银行(北京密云东门支行) 116.84041502475007 40.37451440004574
377 中国农业银行24小时自助银行(北京密云西大桥分理处) 116.82821590039971 40.3750388944487
378 中国农业银行24小时自助银行(顺密旧路) 116.8392577303792 40.36784791250043
379 中国农业银行24小时自助银行服务(北京密云阳光支行) 116.83965363384297 40.362605832186205
380 中国农业银行24小时自

In [58]:
#df_abc_ATM_bj['lng_wgs']=115.97521247335332
#df_abc_ATM_bj['lat_wgs']=40.45921367752729

pd.set_option('precision', 13)

df_abc_ATM_bj['lng_wgs']=lng_wgs
df_abc_ATM_bj['lat_wgs']=lat_wgs

In [59]:
df_abc_ATM_bj[['uid','name','lat','lng','lat_wgs','lng_wgs']]

,uid,name,lat,lng,lat_wgs,lng_wgs
0,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),39.935728,116.439737,116.4270181137492,39.9284094602717
1,8b3c2532c467ed779cf779a3,中国农业银行ATM(和平里东街),39.961913,116.433272,116.4205648041757,39.9544627631169
2,ef2042e0b5c4cc2474026912,中国农业银行ATM(国瑞购物中心),39.904602,116.426825,116.4141495002651,39.8970355478889
3,df0087e9a2747652223bf2e1,中国农业银行ATM(建国门内大街),39.915328,116.426054,116.4133907837023,39.9077518143831
4,d4034eb3f5d52ef6c03644fe,中国农业银行24小时自助银行(北京交道口支行),39.946551,116.415686,116.4030622497937,39.9388425664537
5,93e319afc9e3c02e1e58009b,中国农业银行24小时自助银行(国瑞购物中心),39.905298,116.425725,116.4130548773261,39.8977136106122
6,5b96bda580041957d42ce2c4,中国农业银行24小时自助银行(东单支行),39.916970,116.423978,116.4113246751826,39.9093614589231
7,0603ff0f1707e14c30621293,中国农业银行24小时自助银行服务(北京富贵园分理处),39.900531,116.439997,116.4272571648600,39.8932106573658
8,6f7d8199355ce8cac72ba14a,中国农业银行24小时自助银行服务(崇文支行营业部),39.899935,116.423527,116.4108614233326,39.8923148863504
9,b844bdb44fd2188d2b3a9f89,中国农业银行24小时自助银行(北京和平里东街支行),39.962368,116.431378,116.4186792817527,39.9548818359602


### 获取周边信息
*衣食住行   
*吃喝玩乐   
*政府机构  
*同业  

In [14]:
# 教育培训$政府机构$银行$酒店$美食$生活服务$休闲娱乐$购物
#衣食住行
keywords_1=['购物','美食','酒店','交通设施']
#生活娱乐
keywords_2=['休闲娱乐','生活服务','旅游景点']
#政府医疗金融教育
keywords_3=['政府机构','银行','医疗','教育培训']

keywords_list=[keywords_1,keywords_2,keywords_3]
print(len(keywords_list),keywords_list)
"$".join(keywords_list[0])

3 [['购物', '美食', '酒店', '交通设施'], ['休闲娱乐', '生活服务', '旅游景点'], ['政府机构', '银行', '医疗', '教育培训']]


'购物$美食$酒店$交通设施'

### 配置参数

In [15]:
keywords_str="$".join(keywords_1)
lat=39.935728
lng=116.439737 
radius=1000
ak='10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ'
page_size=20
page_num=0

url_temple='http://api.map.baidu.com/place/v2/search?query={}&location={},{}&radius={}&filter=sort_name:distance|sort_rule:1&page_size={}&page_num={}&output=json&scope=2&ak={}'

url = url_temple.format(
                        keywords_str,
                        lat,
                        lng,
                        radius, 
                        page_size,
                        page_num,
                        ak)
url

'http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=39.935728,116.439737&radius=1000&filter=sort_name:distance|sort_rule:1&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ'

### 定义函数

In [13]:
# 获取结果条数
def get_venues_num_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak):
    url = url_temple.format(
                        keywords_str,
                        lat,
                        lng,
                        radius, 
                        page_size,
                        page_num,
                        ak)
    
    #print(url)
    results=call_api(url)
    venues_num=results['total']
    
    return venues_num     

# 获取查询结果
def get_venues_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak):
    
    #先计算结果总条数
    venues_num = get_venues_num_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)
    
    #用总条数计算结果页数
    page_num_all=venues_num//20 + 1
    print("坐标{},{} 方圆{}米范围内 {}个数：{},总页数{}".format(lat,lng,radius,keywords_str,venues_num,page_num_all))
    
    
    results = pd.DataFrame()
    for num in range(page_num_all):
        #print("完成第{}页".format(num))
        url = url_temple.format(
                        keywords_str,
                        lat,
                        lng,
                        radius, 
                        page_size,
                        page_num,
                        ak)
        
        result = call_api(url)
        #time.sleep(1)
        venues=result['results']
        nearby_venues = json_normalize(venues) # flatten JSON
        #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
        results=pd.concat([results,nearby_venues],ignore_index=True)
        #print(results.shape)
        
    return results


# 目标场所指定关键词 半径搜索
def get_all_around(df,url_temple,keywords_list,lat,lng,radius,page_size,page_num,ak):
    results = pd.DataFrame()
    for index,row in df.iterrows():
        #print(index,row['name'],row['uid'])

        #参数赋值
        keywords_str = "$".join(keywords_list)
        lat=row['lat']
        lng=row['lng']

        #获取结果
        result = get_venues_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)
              
        result['目标_机构名称']=row['name']
        result['目标_uid']=row['uid']
        #合并结果
        results=pd.concat([results,result],ignore_index=True,sort=False)   
    
        print("完成{}-{} {}范围 {}查询,维度：{}".format(index,row['name'],radius,keywords_str,results.shape))
    #处理列名
    results.columns = [col.split(".")[-1] for col in results.columns] 
        
    return results

### 获取周边11类地点信息

In [14]:
#df_abc_ATM_bj.head(2)

In [100]:
print(len(keywords_list),keywords_list)

3 [['购物', '美食', '酒店', '交通设施'], ['休闲娱乐', '生活服务', '旅游景点'], ['政府机构', '银行', '医疗', '教育培训']]


In [101]:
df_bank_around_yszx=get_all_around(df_abc_ATM_bj,url_temple,keywords_1,lat,lng,radius,page_size,page_num,ak)
df_bank_around_yszx.to_csv("data/bj_abc_ATM_bank_around_yszx.csv",index=False)

坐标39.935728000000005,116.43973700000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：186,总页数10
完成中国农业银行24小时自助银行(北京市分行营业部) 1000范围 购物$美食$酒店$交通设施查询,维度：(200, 29)
坐标39.961913,116.433272 方圆1000米范围内 购物$美食$酒店$交通设施个数：174,总页数9
完成中国农业银行ATM(和平里东街) 1000范围 购物$美食$酒店$交通设施查询,维度：(380, 29)
坐标39.904602000000004,116.426825 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行ATM(国瑞购物中心) 1000范围 购物$美食$酒店$交通设施查询,维度：(600, 30)
坐标39.915328,116.42605400000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行ATM(建国门内大街) 1000范围 购物$美食$酒店$交通设施查询,维度：(820, 30)
坐标39.946551,116.41568600000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：194,总页数10
完成中国农业银行24小时自助银行(北京交道口支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(1020, 30)
坐标39.905297999999995,116.425725 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行24小时自助银行(国瑞购物中心) 1000范围 购物$美食$酒店$交通设施查询,维度：(1240, 30)
坐标39.91697,116.423978 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行24小时自助银行(东单支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(1460, 30)
坐标39.900531,116.43999699999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：192,总页数10
完成中国农业银行24小时自助银行服务(北京富贵园分理处) 1000范围 购物$美食$酒店$交通设

坐标39.931101,116.452977 方圆1000米范围内 购物$美食$酒店$交通设施个数：195,总页数10
完成中国农业银行24小时自助银行服务(北京朝阳支行营业部) 1000范围 购物$美食$酒店$交通设施查询,维度：(12751, 31)
坐标39.876733,116.480851 方圆1000米范围内 购物$美食$酒店$交通设施个数：168,总页数9
完成中国农业银行24小时自助银行(北京松榆南路支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(12931, 31)
坐标39.917111,116.60513700000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：164,总页数9
完成中国农业银行24小时自助银行(北京八里桥支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(13111, 31)
坐标39.997518,116.463594 方圆1000米范围内 购物$美食$酒店$交通设施个数：165,总页数9
完成中国农业银行24小时自助银行(北京南湖中园支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(13291, 31)
坐标40.014717,116.47331000000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：166,总页数9
完成中国农业银行ATM(望京商业街(北区)首层望京支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(13471, 31)
坐标40.007923999999996,116.47947099999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行24小时自助银行(北京望京支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(13691, 31)
坐标39.963588,116.47078799999998 方圆1000米范围内 购物$美食$酒店$交通设施个数：186,总页数10
完成中国农业银行24小时自助银行(三元支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(13891, 31)
坐标39.918402,116.479817 方圆1000米范围内 购物$美食$酒店$交通设施个数：196,总页数10
完成中国农业银行24小时自助银行(北京光华路支行) 1000

完成中国农业银行24h自助银行服务(北京紫芳园支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(24431, 31)
坐标39.860867999999996,116.40319 方圆1000米范围内 购物$美食$酒店$交通设施个数：176,总页数9
完成中国农业银行24小时自助银行(北京珠江骏景支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(24611, 31)
坐标39.864912,116.405602 方圆1000米范围内 购物$美食$酒店$交通设施个数：185,总页数10
完成中国农业银行24小时自助银行服务(南苑路辅路) 1000范围 购物$美食$酒店$交通设施查询,维度：(24811, 31)
坐标39.848098,116.38543899999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：179,总页数9
完成中国农业银行24小时自助银行(北京马家堡支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(24991, 31)
坐标39.899733000000005,116.327479 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
完成中国农业银行24小时自助银行(北京西站) 1000范围 购物$美食$酒店$交通设施查询,维度：(25211, 31)
坐标39.814312,116.408023 方圆1000米范围内 购物$美食$酒店$交通设施个数：156,总页数8
完成中国农业银行24小时自助银行(南苑支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(25371, 31)
坐标39.864022,116.289829 方圆1000米范围内 购物$美食$酒店$交通设施个数：158,总页数8
完成中国农业银行24小时自助银行服务(北京丰北支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(25531, 31)
坐标39.851031,116.326523 方圆1000米范围内 购物$美食$酒店$交通设施个数：167,总页数9
完成中国农业银行24小时自助银行服务(万年花城支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(25711, 31)
坐标39.838635,116.29831499999999 方圆1000米范围内 购物$美食$酒店

坐标39.965206,116.29038100000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：159,总页数8
完成中国农业银行24小时自助银行(北京世纪城支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(35593, 31)
坐标39.910992,116.33498700000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：191,总页数10
完成中国农业银行24小时自助银行(北京铁道支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(35793, 31)
坐标39.922852,116.28331899999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：158,总页数8
完成中国农业银行24小时自助银行(北京万寿路支行营业部) 1000范围 购物$美食$酒店$交通设施查询,维度：(35953, 31)
坐标39.936076,116.26068300000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：142,总页数8
完成中国农业银行24小时自助银行(京粮广场) 1000范围 购物$美食$酒店$交通设施查询,维度：(36113, 31)
坐标39.958416,116.367551 方圆1000米范围内 购物$美食$酒店$交通设施个数：187,总页数10
完成中国农业银行24小时自助银行(文慧园路) 1000范围 购物$美食$酒店$交通设施查询,维度：(36313, 31)
坐标40.000384000000004,116.35692900000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：174,总页数9
完成中国农业银行24小时自助银行服务(国家汉办图书资料部西) 1000范围 购物$美食$酒店$交通设施查询,维度：(36493, 31)
坐标40.032505,116.318421 方圆1000米范围内 购物$美食$酒店$交通设施个数：166,总页数9
完成中国农业银行24小时自助银行服务(北京硅谷亮城支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(36673, 31)
坐标40.038221,116.27817399999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：133,总页数7
完成中国农业银行24小时自助银行(

完成中国农业银行24小时自助银行(市场分理处) 1000范围 购物$美食$酒店$交通设施查询,维度：(44852, 31)
坐标40.108207,116.56386200000001 方圆1000米范围内 购物$美食$酒店$交通设施个数：143,总页数8
完成中国农业银行24小时自助银行(裕民大街) 1000范围 购物$美食$酒店$交通设施查询,维度：(45012, 31)
坐标40.224788000000004,116.67858899999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：43,总页数3
完成中国农业银行24小时自助银行(下坡屯路) 1000范围 购物$美食$酒店$交通设施查询,维度：(45072, 31)
坐标40.157584,116.958284 方圆1000米范围内 购物$美食$酒店$交通设施个数：82,总页数5
完成中国农业银行ATM(北京张镇支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(45172, 31)
坐标40.115301,116.66871699999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：157,总页数8
完成中国农业银行24小时自助银行(北京石园分理处) 1000范围 购物$美食$酒店$交通设施查询,维度：(45332, 31)
坐标40.158706,116.837359 方圆1000米范围内 购物$美食$酒店$交通设施个数：115,总页数6
完成中国农业银行ATM(北京杨镇支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(45452, 31)
坐标40.141096000000005,116.58117 方圆1000米范围内 购物$美食$酒店$交通设施个数：74,总页数4
完成中国农业银行24小时自助银行(工业区连通路) 1000范围 购物$美食$酒店$交通设施查询,维度：(45532, 31)
坐标40.121295,116.614346 方圆1000米范围内 购物$美食$酒店$交通设施个数：117,总页数6
完成中国农业银行ATM(北京空港支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(45652, 31)
坐标40.133554,116.584128 方圆1000米范围内 购物$美食$酒店$交通设施个数：32,总页数2
完成中国农业

完成中国农业银行24小时自助银行(北京博兴路支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(53744, 31)
坐标39.808677,116.497801 方圆1000米范围内 购物$美食$酒店$交通设施个数：165,总页数9
完成中国农业银行24小时自助银行(北京北环西路支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(53924, 31)
坐标39.796401,116.349801 方圆1000米范围内 购物$美食$酒店$交通设施个数：155,总页数8
完成中国农业银行24小时自助银行(北京西红门支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(54084, 31)
坐标39.738091,116.338945 方圆1000米范围内 购物$美食$酒店$交通设施个数：162,总页数9
完成中国农业银行24小时自助银行(北京兴华大街支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(54264, 31)
坐标39.737909,116.34661299999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：174,总页数9
完成中国农业银行24小时自助银行(兴丰大街) 1000范围 购物$美食$酒店$交通设施查询,维度：(54444, 31)
坐标39.763346999999996,116.318045 方圆1000米范围内 购物$美食$酒店$交通设施个数：149,总页数8
完成中国农业银行24小时自助银行服务(福海路支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(54604, 31)
坐标39.51639,116.326645 方圆1000米范围内 购物$美食$酒店$交通设施个数：42,总页数3
完成中国农业银行24小时自助银行(北京榆垡支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(54664, 31)
坐标39.79228,116.503799 方圆1000米范围内 购物$美食$酒店$交通设施个数：155,总页数8
完成中国农业银行24小时自助银行(北京荣京西街支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(54824, 31)
坐标39.803545,116.51543899999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：159,

完成中国农业银行24小时自助银行服务(北京延庆区支行营业部) 1000范围 购物$美食$酒店$交通设施查询,维度：(63406, 31)
坐标40.452632,115.991079 方圆1000米范围内 购物$美食$酒店$交通设施个数：147,总页数8
完成中国农业银行24小时自助银行(北京延庆南菜园支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(63566, 31)
坐标40.471994,115.98798899999998 方圆1000米范围内 购物$美食$酒店$交通设施个数：171,总页数9
完成中国农业银行24小时自助银行(高塔分理处) 1000范围 购物$美食$酒店$交通设施查询,维度：(63746, 31)
坐标40.466111,115.981435 方圆1000米范围内 购物$美食$酒店$交通设施个数：166,总页数9
完成中国农业银行24小时自助银行(北京延庆东街支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(63926, 31)
坐标40.466543,115.98578799999999 方圆1000米范围内 购物$美食$酒店$交通设施个数：169,总页数9
完成中国农业银行24小时自助银行(北京延庆东外大街支行) 1000范围 购物$美食$酒店$交通设施查询,维度：(64106, 31)


In [108]:
df_bank_around_shyl=get_all_around(df_abc_ATM_bj,url_temple,keywords_2,lat,lng,radius,page_size,page_num,ak)
df_bank_around_shyl.to_csv("data/bj_abc_ATM_bank_around_shyl.csv",index=False)

坐标39.935728000000005,116.43973700000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：124,总页数7
完成0-中国农业银行24小时自助银行(北京市分行营业部) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(140, 24)
坐标39.961913,116.433272 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：110,总页数6
完成1-中国农业银行ATM(和平里东街) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(260, 24)
坐标39.904602000000004,116.426825 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：132,总页数7
完成2-中国农业银行ATM(国瑞购物中心) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(400, 25)
坐标39.915328,116.42605400000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：132,总页数7
完成3-中国农业银行ATM(建国门内大街) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(540, 25)
坐标39.946551,116.41568600000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：150,总页数8
完成4-中国农业银行24小时自助银行(北京交道口支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(700, 25)
坐标39.905297999999995,116.425725 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：134,总页数7
完成5-中国农业银行24小时自助银行(国瑞购物中心) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(840, 25)
坐标39.91697,116.423978 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：139,总页数7
完成6-中国农业银行24小时自助银行(东单支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(980, 25)
坐标39.900531,116.43999699999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：123,总页数7
完成7-中国农业银行24小时自助银行服务(北京富贵园分理处

完成63-中国农业银行24小时自助银行(北京健德支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8180, 27)
坐标39.976953,116.40888000000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：118,总页数6
完成64-中国农业银行24小时自助银行(北京双北桥支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8300, 27)
坐标39.931101,116.452977 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：143,总页数8
完成65-中国农业银行24小时自助银行服务(北京朝阳支行营业部) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8460, 27)
坐标39.876733,116.480851 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：108,总页数6
完成66-中国农业银行24小时自助银行(北京松榆南路支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8580, 27)
坐标39.917111,116.60513700000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：73,总页数4
完成67-中国农业银行24小时自助银行(北京八里桥支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8660, 27)
坐标39.997518,116.463594 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：87,总页数5
完成68-中国农业银行24小时自助银行(北京南湖中园支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8760, 27)
坐标40.014717,116.47331000000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：106,总页数6
完成69-中国农业银行ATM(望京商业街(北区)首层望京支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(8880, 27)
坐标40.007923999999996,116.47947099999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：102,总页数6
完成70-中国农业银行24小时自助银行(北京望京支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(9000, 27

完成125-中国农业银行24小时自助银行(北京新发地支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15020, 27)
坐标39.893901,116.271615 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：96,总页数5
完成126-中国农业银行24小时自助银行服务(翠微百货) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15120, 27)
坐标39.872583,116.253584 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：48,总页数3
完成127-中国农业银行24小时自助银行服务(小屯西路支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15180, 27)
坐标39.870882,116.44616 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：104,总页数6
完成128-中国农业银行ATM(北京市第二中级人民法院) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15300, 27)
坐标39.868665,116.445912 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：106,总页数6
完成129-中国农业银行24h自助银行服务(北京紫芳园支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15420, 27)
坐标39.860867999999996,116.40319 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：86,总页数5
完成130-中国农业银行24小时自助银行(北京珠江骏景支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15520, 27)
坐标39.864912,116.405602 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：99,总页数5
完成131-中国农业银行24小时自助银行服务(南苑路辅路) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15620, 27)
坐标39.848098,116.38543899999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：107,总页数6
完成132-中国农业银行24小时自助银行(北京马家堡支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(15740, 27)
坐标39.89973300000

完成188-中国农业银行24小时自助银行(北京茉莉园分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(21532, 31)
坐标40.042645,116.214699 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：22,总页数2
完成189-中国农业银行24小时自助银行(太舟坞支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(21572, 31)
坐标39.957534,116.327339 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：124,总页数7
完成190-中国农业银行24小时自助银行(礼堂东路店) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(21712, 31)
坐标39.953036,116.305121 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：102,总页数6
完成191-中国农业银行24小时自助银行服务(北京紫竹桥支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(21832, 31)
坐标39.989401,116.316236 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：129,总页数7
完成192-中国农业银行24小时自助银行(北京北四环支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(21972, 31)
坐标39.969783,116.31505600000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：118,总页数6
完成193-中国农业银行24小时自助银行(北京路西海支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(22092, 31)
坐标39.989785999999995,116.312591 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：130,总页数7
完成194-中国农业银行24小时自助银行(北京海淀支行营业部) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(22232, 31)
坐标39.965206,116.29038100000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：117,总页数6
完成195-中国农业银行24小时自助银行(北京世纪城支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(22352, 31)
坐标3

完成251-中国农业银行24小时自助银行(北京西集分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26469, 31)
坐标39.715665,116.79552199999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：29,总页数2
完成252-中国农业银行24小时自助银行(北京永乐店分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26509, 31)
坐标39.913993,116.642252 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：104,总页数6
完成253-中国农业银行24小时自助银行(北京通州支行营业部) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26629, 31)
坐标39.930211,116.661358 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：22,总页数2
完成254-中国农业银行24小时自助银行(北京通州城关分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26669, 31)
坐标39.9491,116.73836599999998 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：102,总页数6
完成255-中国农业银行24小时自助银行(北京宋庄支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26789, 31)
坐标39.909002,116.68348300000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：75,总页数4
完成256-中国农业银行24小时自助银行(北京通州东关分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26869, 31)
坐标39.914193,116.669936 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：82,总页数5
完成257-中国农业银行24小时自助银行(新华东街) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(26969, 31)
坐标39.91342,116.650558 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：106,总页数6
完成258-中国农业银行24小时自助银行(北京西关分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(27089, 31)
坐标39.808

坐标39.811463,116.433572 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：81,总页数5
完成314-中国农业银行24小时自助银行 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(30731, 31)
坐标39.632209,116.329307 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：31,总页数2
完成315-中国农业银行24小时自助银行 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(30771, 31)
坐标39.727925,116.344302 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：80,总页数5
完成316-中国农业银行24小时自助银行服务(北京博兴路支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(30871, 31)
坐标39.789988,116.427342 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：46,总页数3
完成317-中国农业银行24小时自助银行(北京德茂支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(30931, 31)
坐标39.818706,116.46241599999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：78,总页数4
完成318-中国农业银行24小时自助银行(宣颐路) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(31011, 31)
坐标39.783814,116.44446699999999 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：12,总页数1
完成319-中国农业银行24小时自助银行(德茂街) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(31023, 31)
坐标39.810953999999995,116.447157 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：78,总页数4
完成320-中国农业银行24小时自助银行(北京红星分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(31103, 31)
坐标39.777041,116.34145500000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：84,总页数5
完成321-中国农业银行24小时自助银行(北京郁花园分理处) 1000范围

完成377-中国农业银行24小时自助银行(北京密云西大桥分理处) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35355, 31)
坐标40.374928000000004,116.851826 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：95,总页数5
完成378-中国农业银行24小时自助银行(顺密旧路) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35455, 31)
坐标40.369692,116.852222 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：56,总页数3
完成379-中国农业银行24小时自助银行服务(北京密云阳光支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35515, 31)
坐标40.373362,116.82728200000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：70,总页数4
完成380-中国农业银行24小时自助银行(北京密云双井支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35595, 31)
坐标40.388551,116.871995 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：86,总页数5
完成381-中国农业银行24小时自助银行服务(北京密云行宫支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35695, 31)
坐标40.377426,116.871023 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：63,总页数4
完成382-中国农业银行24小时自助银行(北京密云保利花园支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35775, 31)
坐标40.652104,117.27278700000001 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：101,总页数6
完成383-中国农业银行24小时自助银行(古北驿大街) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35895, 31)
坐标40.480764,116.848977 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：18,总页数1
完成384-中国农业银行24小时自助银行(密云水库支行) 1000范围 休闲娱乐$生活服务$旅游景点查询,维度：(35913, 31)
坐标

In [109]:
df_bank_around_zfjg=get_all_around(df_abc_ATM_bj,url_temple,keywords_3,lat,lng,radius,page_size,page_num,ak)
df_bank_around_zfjg.to_csv("data/bj_abc_ATM_bank_around_zfjg.csv",index=False)

坐标39.935728000000005,116.43973700000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：156,总页数8
完成0-中国农业银行24小时自助银行(北京市分行营业部) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(160, 24)
坐标39.961913,116.433272 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：146,总页数8
完成1-中国农业银行ATM(和平里东街) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(320, 24)
坐标39.904602000000004,116.426825 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：169,总页数9
完成2-中国农业银行ATM(国瑞购物中心) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(500, 25)
坐标39.915328,116.42605400000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：158,总页数8
完成3-中国农业银行ATM(建国门内大街) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(660, 25)
坐标39.946551,116.41568600000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：129,总页数7
完成4-中国农业银行24小时自助银行(北京交道口支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(800, 25)
坐标39.905297999999995,116.425725 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：169,总页数9
完成5-中国农业银行24小时自助银行(国瑞购物中心) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(980, 25)
坐标39.91697,116.423978 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：161,总页数9
完成6-中国农业银行24小时自助银行(东单支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(1160, 25)
坐标39.900531,116.43999699999999 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：162,总页数9
完成7-中国农业银行24小

完成62-中国农业银行24小时自助银行(蜻蜓胡同) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9000, 25)
坐标39.983421,116.390524 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：97,总页数5
完成63-中国农业银行24小时自助银行(北京健德支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9100, 25)
坐标39.976953,116.40888000000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：156,总页数8
完成64-中国农业银行24小时自助银行(北京双北桥支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9260, 25)
坐标39.931101,116.452977 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：171,总页数9
完成65-中国农业银行24小时自助银行服务(北京朝阳支行营业部) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9440, 25)
坐标39.876733,116.480851 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：115,总页数6
完成66-中国农业银行24小时自助银行(北京松榆南路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9560, 25)
坐标39.917111,116.60513700000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：79,总页数4
完成67-中国农业银行24小时自助银行(北京八里桥支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9640, 25)
坐标39.997518,116.463594 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：95,总页数5
完成68-中国农业银行24小时自助银行(北京南湖中园支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9740, 25)
坐标40.014717,116.47331000000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：142,总页数8
完成69-中国农业银行ATM(望京商业街(北区)首层望京支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(9900, 25)
坐标4

完成123-中国农业银行24小时自助银行(北京千禧街支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(16580, 25)
坐标39.867143,116.300068 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：101,总页数6
完成124-中国农业银行24小时自助银行(北京丰台支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(16700, 25)
坐标39.815121000000005,116.34381599999999 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：38,总页数2
完成125-中国农业银行24小时自助银行(北京新发地支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(16740, 25)
坐标39.893901,116.271615 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：103,总页数6
完成126-中国农业银行24小时自助银行服务(翠微百货) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(16860, 25)
坐标39.872583,116.253584 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：57,总页数3
完成127-中国农业银行24小时自助银行服务(小屯西路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(16920, 25)
坐标39.870882,116.44616 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：143,总页数8
完成128-中国农业银行ATM(北京市第二中级人民法院) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(17080, 25)
坐标39.868665,116.445912 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：148,总页数8
完成129-中国农业银行24h自助银行服务(北京紫芳园支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(17240, 25)
坐标39.860867999999996,116.40319 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：105,总页数6
完成130-中国农业银行24小时自助银行(北京珠江骏景支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(17

完成185-中国农业银行24小时自助银行(北京成府路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23534, 25)
坐标39.9943,116.311438 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：120,总页数7
完成186-中国农业银行24小时自助银行(颐和园路) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23674, 25)
坐标40.030738,116.306371 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：69,总页数4
完成187-中国农业银行24小时自助银行(北京农大南路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23754, 25)
坐标40.05571,116.26345800000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：44,总页数3
完成188-中国农业银行24小时自助银行(北京茉莉园分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23814, 25)
坐标40.042645,116.214699 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：32,总页数2
完成189-中国农业银行24小时自助银行(太舟坞支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23854, 25)
坐标39.957534,116.327339 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：134,总页数7
完成190-中国农业银行24小时自助银行(礼堂东路店) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(23994, 25)
坐标39.953036,116.305121 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：110,总页数6
完成191-中国农业银行24小时自助银行服务(北京紫竹桥支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(24114, 25)
坐标39.989401,116.316236 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：166,总页数9
完成192-中国农业银行24小时自助银行(北京北四环支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(24294, 25)
坐标39.969783,11

/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:46: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



完成233-中国农业银行24小时自助银行(北京良乡支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28191, 25)
坐标39.609006,116.035627 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：19,总页数1
完成234-中国农业银行24小时自助银行(北京琉璃河分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28210, 25)
坐标39.708195,115.98791100000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：126,总页数7
完成235-中国农业银行24小时自助银行(北京房山城关支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28350, 25)
坐标39.585681,115.906896 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：39,总页数2
完成236-中国农业银行24小时自助银行(北京长沟支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28390, 25)
坐标39.686671000000004,115.949876 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：24,总页数2
完成237-国农业银行24小时自助银行(北京周口店支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28430, 25)
坐标39.941707,116.709001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：14,总页数1
完成238-中国农业银行ATM(辛疃路) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28444, 25)
坐标39.93018,116.645591 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：25,总页数2
完成239-中国农业银行24小时自助银行服务(物资学院路) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28484, 25)
坐标39.935834,116.68702900000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：47,总页数3
完成240-中国农业银行24小时自助银行 1000范围 政府机构$银行$医疗$教育培训查询,维度：(28544, 25)
坐标39.889708,11

完成295-中国农业银行24小时自助银行服务(天通苑) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(32562, 25)
坐标40.181911,116.392455 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：36,总页数2
完成296-中国农业银行24小时自助银行(北京小汤山分行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(32602, 25)
坐标40.226804,116.25371200000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：117,总页数6
完成297-中国农业银行24小时自助银行(昌平东环分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(32722, 25)
坐标40.225518,116.233081 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：146,总页数8
完成298-中国农业银行24小时自助银行(北京昌平支行营业室) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(32882, 25)
坐标40.151324,116.290873 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：65,总页数4
完成299-中国农业银行24小时自助银行服务(丰善东路) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(32962, 25)
坐标40.064426,116.342541 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：91,总页数5
完成300-中国农业银行24小时自助银行服务(西三旗支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(33062, 25)
坐标40.118393,116.37794 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：43,总页数3
完成301-中国农业银行24小时自助银行服务(宏福分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(33122, 25)
坐标40.237829,116.196168 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：7,总页数1
完成302-中国农业银行24小时自助银行(乐多港奥特莱斯购物中心) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(33129, 25)
坐标40.2338

完成357-中国农业银行24小时自助银行(建设西街) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(37757, 25)
坐标40.192249,117.021694 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：39,总页数2
完成358-中国农业银行24小时自助银行(峪口分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(37797, 25)
坐标40.147094,117.121692 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：143,总页数8
完成359-中国农业银行24小时自助银行(府前街) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(37957, 25)
坐标40.167072,117.13911999999999 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：33,总页数2
完成360-中国农业银行24小时自助银行(北京兴谷开发区支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(37997, 25)
坐标40.14979,117.13031000000001 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：120,总页数7
完成361-中国农业银行24小时自助银行(北京新平东路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(38137, 25)
坐标40.160515000000004,117.12871799999999 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：86,总页数5
完成362-中国农业银行24小时自助银行(北京平谷城关分理处) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(38237, 25)
坐标40.154866,117.117355 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：125,总页数7
完成363-中国农业银行24小时自助银行(北京新平北路支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(38377, 25)
坐标40.150734,117.10649 方圆1000米范围内 政府机构$银行$医疗$教育培训个数：115,总页数6
完成364-中国农业银行24小时自助银行(北京紫贵支行) 1000范围 政府机构$银行$医疗$教育培训查询,维度：(

### 周边数据清洗

#### 加载数据
从本地加载数据，节省调用时间  

In [61]:
df_bank_around_yszx=pd.read_csv("data/bj_ATM_abc_bank_around_yszx.csv")
df_bank_around_shyl=pd.read_csv("data/bj_abc_ATM_bank_around_shyl.csv")
df_bank_around_zfjg=pd.read_csv("data/bj_abc_ATM_bank_around_zfjg.csv")

/home/machinelearning/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3057: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [62]:
print(df_bank_around_shyl.shape)
df_bank_around_shyl.info()

(36713, 31)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36713 entries, 0 to 36712
Data columns (total 31 columns):
address               36713 non-null object
area                  36713 non-null object
city                  36713 non-null object
detail                36713 non-null float64
children              36713 non-null object
comment_num           14542 non-null float64
detail_url            36713 non-null object
distance              36713 non-null float64
image_num             30057 non-null float64
lat                   10290 non-null float64
lng                   10290 non-null float64
overall_rating        17667 non-null float64
price                 3962 non-null object
tag                   36616 non-null object
type                  36703 non-null object
lat.1                 36713 non-null float64
lng.1                 36713 non-null float64
name                  36713 non-null object
province              36713 non-null object
street_id             22339 non-nul

In [63]:
#df_bank_around_zfjg.head()

In [64]:
df_bank_around_yszx.columns = [col.split(".")[-1] for col in df_bank_around_yszx.columns]

In [65]:
df_bank_around_all = pd.DataFrame()
sel_columns= ['目标_uid','目标_机构名称','name','uid','distance','type','tag']
#衣食住行
df_tmp = df_bank_around_yszx[sel_columns]
df_bank_around_all = pd.concat([df_bank_around_all,df_tmp],ignore_index=True,sort=False)   

#生活娱乐
df_tmp = df_bank_around_shyl[sel_columns]
df_bank_around_all = pd.concat([df_bank_around_all,df_tmp],ignore_index=True,sort=False)  


#政府机构教育医疗
df_tmp = df_bank_around_zfjg[sel_columns]
df_bank_around_all = pd.concat([df_bank_around_all,df_tmp],ignore_index=True,sort=False) 


In [66]:
def split_1(tag):
    try:
        type_str = tag.split(";")[0]
    except:
        type_str=tag
    return type_str

def split_2(tag):
    try:
        type_str = tag.split(";")[-1]
    except:
        type_str=tag
    return type_str

In [67]:
# 删除有空值的行，因为空值较少 所以直接删除
df_bank_around_all.dropna(how='any',inplace=True)

# 从tag拆分出 商铺的一级分类
df_bank_around_all['一级行业分类']=df_bank_around_all['tag'].apply(split_1)

# 从tag拆分出 商铺的二级分类
df_bank_around_all['二级行业分类']=df_bank_around_all['tag'].apply(split_2)

In [68]:
df_bank_around_all.sample(5)

,目标_uid,目标_机构名称,name,uid,distance,type,tag,一级行业分类,二级行业分类
86921,251241fef7e9ac32940319b6,中国农业银行24小时自助银行服务(国家汉办图书资料部西),中国邮政(学院路邮政支局),0523a14174d64a1db23c8142,147.0,life,生活服务;邮局,生活服务,邮局
118780,2c0bd6c5c85524aa43ab9add,中国农业银行24小时自助银行服务(万年花城支行),中国农业银行(万年花城支行),7096a9a30fdf97994251c0f0,16.0,life,金融;银行,金融,银行
11195,a0978bbceba66185da2f28b6,中国农业银行24小时自助银行(北京小营北路支行),西北郎中国兰州牛肉拉面(北京小营路店),b8635da37afe48893062123d,97.0,cater,美食;小吃快餐店,美食,小吃快餐店
31494,10b39a8b6ea901d941e19246,中国农业银行24小时自助银行(资源大厦西北),3W咖啡,bc4dce2e0cd24924a1a934cf,108.0,cater,美食;咖啡厅,美食,咖啡厅
76120,f3c2373cf4d5692fd22dd61e,中国农业银行24小时自助银行服务(北京康运支行),小陈制衣,07842706504556493a04b196,156.0,life,生活服务;其他,生活服务,其他


## 特征工程

In [69]:
# 周边商铺的所有 一级行业分类
print(df_bank_around_all['一级行业分类'].nunique(),df_bank_around_all['一级行业分类'].unique())

# 周边商铺的所有 二级行业分类
print(df_bank_around_all['二级行业分类'].nunique(),df_bank_around_all['二级行业分类'].unique())

19 ['交通设施' '美食' '道路' '酒店' '购物' '出入口' '公司企业' '医疗' '生活服务' '房地产' '教育培训' '行政地标'
 '休闲娱乐' '文化传媒' '旅游景点' '政府机构' '汽车服务' '丽人' '金融']
116 ['停车场' '茶座' '路口' '快捷酒店' '路侧停车位' '超市' '市场' '中餐厅' '商铺' '蛋糕甜品店' '小吃快餐店' '便利店'
 '外国餐厅' '购物中心' '其他' '咖啡厅' '美食' '家电数码' '停车场出入口' '家居建材' '充电站' '购物' '公司' '酒吧'
 '星级酒店' '加油加气站' '桥' '公寓式酒店' '医疗器械' '洗衣店' '宠物服务' '港口' '住宅区' '培训机构' '民宿'
 '村庄' '火车站' '医疗保健' '度假村' '农家院' '酒店' '百货商场' '长途汽车站' '剧院' '美术馆' '图文快印店'
 '房产中介机构' '洗浴按摩' '休闲广场' '物流公司' '公园' '博物馆' '教堂' '文物古迹' '邮局' 'ktv' '文化宫'
 '展览馆' '公共厕所' '公用事业' '游戏场所' '照相馆' '游乐园' '维修点' '通讯营业厅' '售票处' '生活服务' '风景区'
 '网吧' '彩票销售点' '家政服务' '电影院' '歌舞厅' '景点' '旅游景点' '汽车维修' '动物园' '殡葬服务' '报刊亭'
 '寺庙' '休闲娱乐' '诊所' '涉外机构' '科技馆' '美容' '汽车美容' '行政单位' '医疗' '植物园' '写字楼' '疾控中心'
 '美体' 'atm' '银行' '药店' '公检法机构' '中央机构' '社会团体' '各级政府' '党派团体' '政府机构' '体检机构'
 '居民委员会' '信用社' '科研机构' '福利机构' '综合医院' '投资理财' '政治教育机构' '疗养院' '金融' '专科医院'
 '民主党派' '急救中心' '教育培训' '农林园艺']


### 按商铺一级行业分类 汇总统计

In [70]:
# 每个分类的总个数
pivt_tmp = pd.pivot_table(
        df_bank_around_all,values=['uid'],
        index=['目标_uid'],columns=['一级行业分类'],
        aggfunc=['count'],
        fill_value=0)


pivt_tmp.columns = ['_'.join(col).strip() for col in pivt_tmp.columns.ravel()]
pivt_tmp.reset_index(inplace=True)
df_type1_count = pivt_tmp

In [71]:
# 每个分类的距离的最小值、平均值、总和
pivt_tmp = pd.pivot_table(
        df_bank_around_all,values=['distance'],
        index=['目标_uid'],columns=['一级行业分类'],
        aggfunc=['min','mean','sum'],
        fill_value=-1)


pivt_tmp.columns = ['_'.join(col).strip() for col in pivt_tmp.columns.ravel()]
pivt_tmp.reset_index(inplace=True)
df_distance_pivt=pivt_tmp

In [72]:
df_features = df_type1_count.merge(df_distance_pivt,on='目标_uid',how='outer')

In [73]:
df_features.head()

,目标_uid,count_uid_丽人,count_uid_交通设施,count_uid_休闲娱乐,count_uid_公司企业,count_uid_出入口,count_uid_医疗,count_uid_房地产,count_uid_政府机构,count_uid_教育培训,...,sum_distance_文化传媒,sum_distance_旅游景点,sum_distance_汽车服务,sum_distance_生活服务,sum_distance_美食,sum_distance_行政地标,sum_distance_购物,sum_distance_道路,sum_distance_酒店,sum_distance_金融
0,00d8bdb5cd0c1640dbeced23,0,40,15,5,0,16,0,88,0,...,955,-1,-1,14090,3350,-1,9310,2980,-1,6024
1,00f1c7fb2398311fc156acbc,0,88,4,0,8,11,4,77,0,...,1848,-1,-1,15944,2848,-1,1368,7448,-1,5929
2,00f1c7fb49ca6b62c056ac65,0,36,12,0,0,42,0,6,0,...,2652,-1,-1,8722,3750,-1,11142,2040,9924,5250
3,027ffd33546e62c1fda7780d,0,9,10,14,0,6,0,96,0,...,235,-1,-1,9175,4707,-1,1917,-1,-1,1752
4,0357627716d0251f374aae7e,0,99,14,7,0,0,0,54,9,...,-1,4550,-1,10878,3281,-1,3201,3377,-1,11601


## 建模

In [74]:
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [75]:
# set number of clusters
kclusters = 5

bank_ATM_grouped_clustering = df_features.drop(columns=['目标_uid'])

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=2509).fit(bank_ATM_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 4, 3, 2, 3, 2, 0, 1, 1, 3], dtype=int32)

In [76]:
ATM_cluster=df_features[['目标_uid']]
ATM_cluster.columns=['uid']
ATM_cluster['label']=kmeans.labels_

/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [77]:
ATM_cluster

,uid,label
0,00d8bdb5cd0c1640dbeced23,3
1,00f1c7fb2398311fc156acbc,4
2,00f1c7fb49ca6b62c056ac65,3
3,027ffd33546e62c1fda7780d,2
4,0357627716d0251f374aae7e,3
5,03a167c3591d5a8729fc489f,2
6,03bfe83739b446cb4207c1e2,0
7,0494cb328054b6442fb908ff,1
8,0523a141e681d81eb33c81a9,1
9,0545314ba3ee7f4ecae33c84,3


## 结果分析

### 加标签

In [78]:
bj_ATM_abc = df_abc_ATM_bj.merge(ATM_cluster,how='left',on='uid')
#bj_ATM_abc.drop(columns=['detail','telephone'],inplace=True)
bj_ATM_abc = bj_ATM_abc[['uid','name','label','lat_wgs','lng_wgs','area']]

In [79]:
bj_ATM_abc.sample(10)

,uid,name,label,lat_wgs,lng_wgs,area
276,2fd2beab81b0e22c7adbd25b,中国农业银行24小时自助银行(顺义支行营业部),2,116.6449862352146,40.1285731494128,顺义区
68,82986ffc3507be97223bf28c,中国农业银行24小时自助银行(北京南湖中园支行),4,116.4507878095431,39.9905139000787,朝阳区
110,357ea2ffd822f0d5a6da3999,中国农业银行24小时自助银行,1,116.3001460611083,39.8651791267207,丰台区
376,cb4fd146bfe4e53f9d9d6873,中国农业银行24小时自助银行(北京密云东门支行),3,116.8404150247501,40.3745144000457,密云区
33,cc2991bc4c0a0bc203ff2d60,中国农业银行24小时自助银行(北京金融大街支行),3,116.3547834524118,39.9169268027502,西城区
297,e43340df3b0313dcbda2407b,中国农业银行24小时自助银行(昌平东环分理处),0,116.2411080948017,40.2196098891504,昌平区
221,ed37761d55debfff0f0a7041,中国农业银行24h自助银行服务,3,116.0635665960068,39.6506142207765,房山区
286,39c93770a7d42c401df74d66,中国农业银行ATM(环陵路),1,116.2218609782375,40.2931056986833,昌平区
113,e9be6a4e219fee1e04fef56f,中国农业银行24小时自助银行服务(东方友谊商务楼东南),2,116.3342526458570,39.8495264231110,丰台区
258,9a99869c3bb5d663da2f28cb,中国农业银行24小时自助银行(北京西关分理处),1,116.6383294055047,39.9059996800258,通州区


### 地图展示

In [80]:
bj_ATM_abc.sample(10)

,uid,name,label,lat_wgs,lng_wgs,area
171,328f7d0b06ec5606d9d89189,中国农业银行24小时自助银行(北京地质大学分理处),4,116.3436118525093,39.9903131914996,海淀区
17,e6e5cdd52a45fd695cbe86c5,中国农业银行ATM,4,116.3504455653155,39.9397875888189,西城区
43,243dea6807cc1a6517c8f67a,中国农业银行24小时自助银行服务(万意奥特莱斯西南),3,116.4069211402405,40.0549994563502,朝阳区
248,a31aa9be87176814fd0eb0a1,中国农业银行24小时自助银行(北京梨园支行),1,116.6600455700385,39.8789641603695,通州区
285,edd64ce1a66799913fe23163,中国农业银行24小时自助银行(北京回龙观支行),3,116.3199793344722,40.0763063034677,昌平区
230,dffd4642476e22b13d9a5eb9,中国农业银行24小时自助银行(河北支行),1,115.9302997278231,39.8159066021537,房山区
207,b388f9fd01234c41b25c6aa3,中国农业银行24小时自助银行(北京永通支行),4,116.2583438274598,39.9127740603319,海淀区
14,ffd6d4030f55718f17b54bfe,中国农业银行24小时自助银行(北京东四北支行),1,116.4109604977174,39.9277410617355,东城区
243,75384cc693bfa686ce04efaa,中国农业银行24小时自助银行(北京联华分理处),0,116.6369922767586,39.8920825526872,通州区
246,6af17ea94c97ab7aa2d23064,中国农业银行24小时自助银行(康定街网点),1,116.5340776555319,39.7714272554740,通州区


In [154]:
from pyecharts.charts import Geo,BMap
from pyecharts.faker import Collector, Faker
from pyecharts import options as opts
from pyecharts.globals import GeoType

from pyecharts.render import make_snapshot

from snapshot_phantomjs import snapshot

In [155]:
def make_geo(city,df_atm):        
    g = Geo()
    g.add_schema(maptype=city,zoom=1.2)
    
    # 定义数据对，
    data_pair = []
    
    # 定义坐标对应的名称，添加到坐标库中 add_coordinate(name, lng, lat)
    for index,row in df_atm.iterrows():
        g.add_coordinate(row['name'], row['lat_wgs'], row['lng_wgs'])
        
        data_pair.append((row['name'],row['label']))     
        
    

    # Geo 图类型，有 scatter, effectScatter, heatmap, lines 4 种，建议使用
    # from pyecharts.globals import GeoType
    # GeoType.GeoType.EFFECT_SCATTER，GeoType.HEATMAP，GeoType.LINES

    # 将数据添加到地图上
    g.add('', data_pair, type_=GeoType.EFFECT_SCATTER, symbol_size=5)
    # 设置样式
    g.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    pieces = [
        {'max': 0, 'label': '0', 'color': '#50A3BA'},
        {'min': 1, 'max': 1, 'label': '1', 'color': '#3700A4'},
        {'min': 2, 'max': 2, 'label': '2', 'color': '#81AE9F'},
        {'min': 3, 'max': 3, 'label': '3', 'color': '#E2C568'},
        {'min': 4, 'max': 4, 'label': '4', 'color': '#FCF84D'},
        {'min': 5, 'max': 5, 'label': '5', 'color': '#DD0200'},
        {'min': 6, 'max': 6, 'label': '6', 'color': '#DD675E'},
        {'min': 7, 'label': '7', 'color': '#D94E5D'}  # 有下限无上限
    ]
    #  is_piecewise 是否自定义分段， 变为true 才能生效
    g.set_global_opts(
        visualmap_opts=opts.VisualMapOpts(is_piecewise=True, pieces=pieces),
        title_opts=opts.TitleOpts(title="{}-ATM聚类分布".format(city)),
    )
    return g    

In [156]:
geo=make_geo("北京",bj_ATM_abc)

In [158]:
#make_snapshot(snapshot, geo.render(), "data/atm_bj.png")
geo.render_notebook()

In [149]:
bj_ATM_abc['label'].value_counts()

1    117
2    105
3     94
0     45
4     35
Name: label, dtype: int64

### 百度地图

In [145]:
from pyecharts.globals import BMapType

In [146]:
BAIDU_AK = "EvD2vT2r4LXZXaeIT0PmFoNObeqcRUle"

def bmap_base() -> BMap:
    c = (
        BMap()
        .add_schema(baidu_ak=BAIDU_AK, center=[120.13066322374, 30.240018034923])
        .add(
            "bmap",
            [list(z) for z in zip(Faker.provinces, Faker.values())],
            label_opts=opts.LabelOpts(formatter="{b}"),
        )
        .set_global_opts(title_opts=opts.TitleOpts(title="BMap-基本示例"))
    )
    return c

In [147]:
def make_bmap(city,df_atm):        
    g = BMap()
    g.add_schema(
        baidu_ak=BAIDU_AK,
            center=[120.13066322374, 30.240018034923])
    
    # 定义数据对，
    data_pair = []
    
    # 定义坐标对应的名称，添加到坐标库中 add_coordinate(name, lng, lat)
    for index,row in df_atm.iterrows():
        g.add_coordinate(row['name'], row['lat_wgs'], row['lng_wgs'])
        
        data_pair.append((row['name'],row['label']))     
        
    

    # Geo 图类型，有 scatter, effectScatter, heatmap, lines 4 种，建议使用
    # from pyecharts.globals import GeoType
    # GeoType.GeoType.EFFECT_SCATTER，GeoType.HEATMAP，GeoType.LINES

    # 将数据添加到地图上
    g.add('bmap', data_pair, label_opts=opts.LabelOpts(formatter="{b}"))
    # 设置样式
    g.set_global_opts(title_opts=opts.TitleOpts(title="{}-ATM聚类分布".format(city)))
    
    
    return g    

In [148]:
#atm_map=make_bmap("北京",bj_ATM_abc)
atm_map = bmap_base()
#atm_map
atm_map.render_notebook()

In [142]:
df_bank_around_all[df_bank_around_all['tag'].isnull()]

,目标_uid,目标_机构名称,name,uid,distance,type,tag,标点类型


In [143]:
df_bank_around_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 142156 entries, 0 to 142331
Data columns (total 8 columns):
目标_uid      142156 non-null object
目标_机构名称     142156 non-null object
name        142156 non-null object
uid         142156 non-null object
distance    142156 non-null float64
type        142156 non-null object
tag         142156 non-null object
标点类型        142156 non-null object
dtypes: float64(1), object(7)
memory usage: 9.8+ MB


In [130]:
sel_columns= ['目标_uid','目标_机构名称','name','uid','detail_info.distance','detail_info.type','detail_info.tag']
sel_columns_1= ['目标_uid','目标_机构名称','name','uid','distance','type','tag']
df_tmp = df_bank_around_zfjg.copy()
df_tmp[sel_columns_1]

,目标_uid,目标_机构名称,name,uid,distance,type,tag
0,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),中国农业银行24小时自助银行(北京市分行营业部),40b24ff63ae969b4d518b7b5,0,life,金融;atm
1,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),中国农业银行(北京市分行),d45be3778d313efc2d473303,44,life,金融;银行
2,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),北京月儒堂中医诊所,2350d3de4392a7ef4f3a5974,66,hospital,医疗;诊所
3,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),北京东洲中医门诊部,f5b184521fef60ea7adbd220,80,hospital,医疗;其他
4,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),中信银行(北京南新仓支行),6c52fdae8d5d8a6d047ce8bc,109,life,金融;银行
5,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),九芝堂(北京店),4cec3d24e2eb48f6fb22130d,122,hospital,医疗;药店
6,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),中国银行(第五广场支行),4fc3161a6def52aa3b13fda1,154,life,金融;银行
7,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),中国银行24小时自助银行服务(北京第五广场支行),90c333aa0a8bb9e4c1f1cfb0,161,life,金融;atm
8,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),交通警察,251935437b83324e57bfffab,163,life,政府机构;行政单位
9,40b24ff63ae969b4d518b7b5,中国农业银行24小时自助银行(北京市分行营业部),瑞弛基金,ccaaef3f5dce3779d22dd6c0,170,life,政府机构;行政单位


In [82]:
results = pd.DataFrame()
for index,row in df_abc_ATM_bj_tmp.iterrows():
    print(index,row['name'],row['uid'])
    
    #参数赋值
    keywords_str = "$".join(keywords_1)
    lat=row['lat']
    lng=row['lng']
    
    #获取结果
    result = get_venues_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)
    #处理列名
    #result.columns = [col.split(".")[-1] for col in result.columns]       
    result['目标_机构名称']=row['name']
    result['目标_uid']=row['uid']
    #合并结果
    results=pd.concat([results,result],ignore_index=True,sort=False)   

153 中国农业银行24小时自助银行 6019856784c2ba86c27da2c9
http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=40.011738,116.289197&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标40.011738,116.289197 方圆1000米范围内 购物$美食$酒店$交通设施个数：136,总页数7
94 中国农业银行24小时自助银行(北京十里河分理处) 9191ba13002148ddab059ded
http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=39.871512,116.468694&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标39.871512,116.468694 方圆1000米范围内 购物$美食$酒店$交通设施个数：200,总页数11
378 中国农业银行24小时自助银行(顺密旧路) 1dafe589e896b8366c476031
http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=40.374928000000004,116.851826&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标40.374928000000004,116.851826 方圆1000米范围内 购物$美食$酒店$交通设施个数：172,总页数9
141 中国农业银行24小时自助银行服务(北京苹果园支行) ae4122ebf5a34df98e7727b3
http://api.map.baidu.com/place/v2/search?query=购物$美食$

In [83]:
results

,address,area,city,detail,detail_info.children,detail_info.comment_num,detail_info.detail_url,detail_info.distance,detail_info.navi_location.lat,detail_info.navi_location.lng,...,location.lng,name,province,street_id,telephone,uid,目标_机构名称,目标_uid,detail_info.image_num,detail_info.indoor_floor
0,蜈蚣街1号,海淀区,北京市,1,[],2,http://api.map.baidu.com/place/detail?uid=af1a...,50,40.011556,116.289418,...,116.289546,玉泉连锁超市(党校店),北京市,af1ae56bd282e4114207c17d,NaN,af1ae56bd282e4114207c17d,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
1,大有庄南上坡2号,海淀区,北京市,1,[],3,http://api.map.baidu.com/place/detail?uid=0a2c...,385,40.008130,116.289316,...,116.289381,红美超市(宫门前街),北京市,0a2c6c956584c74e100daa3d,NaN,0a2c6c956584c74e100daa3d,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
2,颐和园路99号,海淀区,北京市,1,[],NaN,http://api.map.baidu.com/place/detail?uid=8720...,372,NaN,NaN,...,116.287800,三福城,北京市,8720683cce4c900e3310afce,NaN,8720683cce4c900e3310afce,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
3,北京市海淀区颐和园路100号,海淀区,北京市,1,[],1,http://api.map.baidu.com/place/detail?uid=b2d8...,409,NaN,NaN,...,116.286862,酒茶生(中央党校店),北京市,NaN,NaN,b2d86eb9d94eabff81a12ef2,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
4,北京市海淀区颐和园路100号,海淀区,北京市,1,[],7,http://api.map.baidu.com/place/detail?uid=3a86...,414,NaN,NaN,...,116.286821,酒美乐国际名酒城,北京市,3a8620f7cc061407770269fc,(010)62809060,3a8620f7cc061407770269fc,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
5,北京市海淀区大有庄100号中共中央党校内,海淀区,北京市,1,[],NaN,http://api.map.baidu.com/place/detail?uid=c7c1...,546,NaN,NaN,...,116.282783,大有书局,北京市,NaN,NaN,c7c10660d514c17d1fe9993e,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
6,北京市海淀区东学路与月华路交叉口西50米,海淀区,北京市,1,[],NaN,http://api.map.baidu.com/place/detail?uid=9fd8...,294,NaN,NaN,...,116.292277,好邻居,北京市,NaN,NaN,9fd8579718f993a7a1d23077,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
7,大有庄大街与蜈蚣街交叉口东北150米,海淀区,北京市,1,[],1,http://api.map.baidu.com/place/detail?uid=7f26...,138,NaN,NaN,...,116.290325,骚子营社区-便利超市,北京市,7f26f1bb72ba0d2f9d82ea6b,NaN,7f26f1bb72ba0d2f9d82ea6b,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
8,北京市海淀区宫门前街1号35幢等9幢--36幢平房,海淀区,北京市,1,[],5,http://api.map.baidu.com/place/detail?uid=23bd...,760,NaN,NaN,...,116.288699,快客便利(宫门前街店),北京市,23bd1b0b4b56ec48c42ba13c,(010)82161337,23bd1b0b4b56ec48c42ba13c,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN
9,北京市海淀区哨子营100号,海淀区,北京市,1,[],1,http://api.map.baidu.com/place/detail?uid=0a86...,743,40.017469,116.293163,...,116.293466,伟丰烟酒城,北京市,0a86312957c7dc6beee4a5ca,NaN,0a86312957c7dc6beee4a5ca,中国农业银行24小时自助银行,6019856784c2ba86c27da2c9,NaN,NaN


In [69]:
results = pd.DataFrame()
for key in keywords_list:
    keywords_str = "$".join(key)
    print(keywords_str)
    
    #获取结果
    result = get_venues_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)
    #处理列名
    result.columns = [col.split(".")[-1] for col in result.columns]       
    result['机构名称']=keyword
    #合并结果
    results=pd.concat([results,result],ignore_index=True,sort=False)    
    

购物$美食$酒店$交通设施
http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=39.935728,116.439737&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标39.935728,116.439737 方圆1000米范围内 购物$美食$酒店$交通设施个数：186,总页数10
休闲娱乐$生活服务$旅游景点
http://api.map.baidu.com/place/v2/search?query=休闲娱乐$生活服务$旅游景点&location=39.935728,116.439737&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标39.935728,116.439737 方圆1000米范围内 休闲娱乐$生活服务$旅游景点个数：124,总页数7


AssertionError: Number of manager items must equal union of block items
# manager items: 24, # tot_items: 26

In [63]:
#get_venues_num_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)
get_venues_1(url_temple,keywords_str,lat,lng,radius,page_size,page_num,ak)

http://api.map.baidu.com/place/v2/search?query=购物$美食$酒店$交通设施&location=39.935728,116.439737&radius=1000&page_size=20&page_num=0&output=json&scope=2&ak=10ykqahdsmqaYOkrLgPzNNc6QAD2HOSZ
坐标39.935728,116.439737 方圆1000米范围内 购物$美食$酒店$交通设施个数：186,总页数10


,address,area,city,detail,detail_info.children,detail_info.comment_num,detail_info.detail_url,detail_info.distance,detail_info.navi_location.lat,detail_info.navi_location.lng,...,detail_info.price,detail_info.tag,detail_info.type,location.lat,location.lng,name,province,street_id,telephone,uid
0,北京市朝阳区朝阳门外大街18号,朝阳区,北京市,1,[],47,http://api.map.baidu.com/place/detail?uid=66af...,824,39.929113,116.444491,...,378.0,购物;购物中心,shopping,39.929555,116.445090,丰联广场,北京市,66af0ae5a237d4fff6eafccd,(010)65881997,66af0ae5a237d4fff6eafccd
1,北京市朝阳区吉庆里10号,朝阳区,北京市,1,[],NaN,http://api.map.baidu.com/place/detail?uid=39d4...,573,39.932784,116.445578,...,NaN,房地产;写字楼,house,39.932998,116.445437,蓝筹名座,北京市,39d43f827bc5ce86a9059d8f,NaN,39d43f827bc5ce86a9059d8f
2,北京市东城区朝阳门内大街8号朝阳首府地下一层,东城区,北京市,1,[],43,http://api.map.baidu.com/place/detail?uid=f4c0...,671,NaN,NaN,...,0.0,购物;超市,shopping,39.930013,116.437179,物美超市(朝阳门店),北京市,f4c0581121684d6b4c6d46da,(010)57929132,f4c0581121684d6b4c6d46da
3,北京市东城区新中西里26号,东城区,北京市,1,[],39,http://api.map.baidu.com/place/detail?uid=96b6...,689,39.939717,116.445100,...,0.0,购物;超市,shopping,39.940005,116.445594,物美超市(新兴里店),北京市,96b672aa5387476cce04ef64,(010)64175592,96b672aa5387476cce04ef64
4,东城区朝阳门北大街8号富华大厦d座一层1-a,东城区,北京市,1,[],41,http://api.map.baidu.com/place/detail?uid=2167...,256,39.936328,116.442690,...,NaN,购物;便利店,shopping,39.936375,116.442627,罗森便利店(富华大厦店),北京市,21677f639110a4a8fb22130d,4008206390,21677f639110a4a8fb22130d
5,北京市东城区豆瓣胡同1号楼B1楼,东城区,北京市,1,[],24,http://api.map.baidu.com/place/detail?uid=edd6...,148,39.935364,116.438116,...,0.0,购物;超市,shopping,39.935122,116.438182,顺天府超市(豆瓣胡同店),北京市,edd64ce130b46f883ee231aa,"(010)84065294,(010)84066871",edd64ce130b46f883ee231aa
6,东城区豆瓣胡同1号楼,东城区,北京市,1,[],10,http://api.map.baidu.com/place/detail?uid=aa4a...,130,39.935063,116.438525,...,0.0,购物;超市,shopping,39.935063,116.438480,世纪华联超市(豆瓣胡同店),北京市,aa4ac95b71d023e78c1a8957,13051436521,aa4ac95b71d023e78c1a8957
7,工体北路巨石大厦B1层,东城区,北京市,1,[],NaN,http://api.map.baidu.com/place/detail?uid=dc20...,695,NaN,NaN,...,NaN,购物;市场,shopping,39.939851,116.445866,小商品批发市场,北京市,NaN,NaN,dc20b63ec80cd833a8f72931
8,东城区工体北路66号瑞士公寓1号楼80102-80103号,东城区,北京市,1,[],13,http://api.map.baidu.com/place/detail?uid=5b25...,519,39.939451,116.443695,...,26300.0,购物;商铺,shopping,39.939276,116.443695,柏斯琴行(东四十条店),北京市,5b25f44624421af3112935a2,(010)65513729,5b25f44624421af3112935a2
9,北京市东城区朝阳门内北小街105号,东城区,北京市,1,[],20,http://api.map.baidu.com/place/detail?uid=7fd7...,648,39.934573,116.432463,...,3926.0,购物;商铺,shopping,39.934555,116.432285,捷安特专卖店(北小街店),北京市,7fd72311e69636865cbe8677,(010)64041924,7fd72311e69636865cbe8677


In [55]:
while True:
    try:
        results = requests.get(url).json()
        if results['status']==0:
            break
    except:
        print("调用接口失败")

In [62]:
get_venues_num(keyword,tag,region,page_size,page_num,ak)

19

In [46]:
#results = requests.get(url).json()
results

{'status': 0,
 'message': 'ok',
 'total': 21,
 'results': [{'name': '中国农业银行(北京怀柔东环支行)',
   'location': {'lat': 40.33447, 'lng': 116.650679},
   'address': '北京市怀柔区开放路99号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'b9e5d0b8344d39bc81a12e72',
   'telephone': '(010)69641626',
   'detail': 1,
   'uid': 'b9e5d0b8344d39bc81a12e72'},
  {'name': '中国农业银行(北京城北支行)',
   'location': {'lat': 40.340249, 'lng': 116.63708},
   'address': '北京市怀柔区富乐小区北里25号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'd1469ee75f133e6384330474',
   'telephone': '(010)69645710',
   'detail': 1,
   'uid': 'd1469ee75f133e6384330474'},
  {'name': '中国农业银行(北京湖光支行)',
   'location': {'lat': 40.332219, 'lng': 116.637282},
   'address': '北京市怀柔区湖光小区青春路甲54号',
   'province': '北京市',
   'city': '北京市',
   'area': '怀柔区',
   'street_id': 'e0f871002a2feacfc16c144d',
   'telephone': '(010)69643717',
   'detail': 1,
   'uid': 'e0f871002a2feacfc16c144d'},
  {'name': '中国农业银行(北京红螺支行)

In [42]:
results['status']

0

In [35]:
venues_num=results['total']
page_num_all=venues_num//20 + 1
print("{} 区域内,{}-{}个数：{},总页数{}".format(region,keyword,tag,venues_num,page_num_all))
#venues=results['results']

怀柔区 区域内,农业银行-金融个数：21,总页数2


In [37]:
results = pd.DataFrame()
for num in range(page_num_all):
    print("完成第{}页".format(num))
    url = 'http://api.map.baidu.com/place/v2/search?query={}&tag={}&region={}&page_size={}&page_num={}&output=json&ak={}'.format(
       keyword,
       tag,
       region,
       page_size,
       num,
       ak)
    result = requests.get(url).json()
    time.sleep(5)
    venues=result['results']
    nearby_venues = json_normalize(venues) # flatten JSON
    #nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
    results=pd.concat([results,nearby_venues],ignore_index=True)
    print(results.shape)
    

完成第0页
(20, 11)
完成第1页
(21, 11)


/home/machinelearning/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [38]:
results.columns = [col.split(".")[-1] for col in results.columns]

In [39]:
results

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市怀柔区开放路99号,怀柔区,北京市,1,40.334470,116.650679,中国农业银行(北京怀柔东环支行),北京市,b9e5d0b8344d39bc81a12e72,(010)69641626,b9e5d0b8344d39bc81a12e72
1,北京市怀柔区富乐小区北里25号,怀柔区,北京市,1,40.340249,116.637080,中国农业银行(北京城北支行),北京市,d1469ee75f133e6384330474,(010)69645710,d1469ee75f133e6384330474
2,北京市怀柔区湖光小区青春路甲54号,怀柔区,北京市,1,40.332219,116.637282,中国农业银行(北京湖光支行),北京市,e0f871002a2feacfc16c144d,(010)69643717,e0f871002a2feacfc16c144d
3,富乐大街26号院11,怀柔区,北京市,1,40.347739,116.637551,中国农业银行(北京红螺支行),北京市,92b806d2ad46ec8379b55c20,(010)89688975,92b806d2ad46ec8379b55c20
4,北京市怀柔区庙城镇庙城村510号1幢1至2层,怀柔区,北京市,1,40.299453,116.634530,中国农业银行(北京庙城支行),北京市,c7eca12b84e69936e115f93e,(010)60693257,c7eca12b84e69936e115f93e
5,北京市怀柔区府前东街12号楼7至8号,怀柔区,北京市,1,40.322605,116.647051,中国农业银行(北京迎宾支行),北京市,2bcbc54cdb894e3c2202208b,95599,0ce65c2d6b6f7d592d473372
6,北京市怀柔区金台园甲56号,怀柔区,北京市,1,40.327879,116.645463,中国农业银行(北京兴怀大街支行),北京市,442076466958d516deee8b23,(010)69644532,442076466958d516deee8b23
7,北京市怀柔区,怀柔区,北京市,1,40.415641,116.695159,中国农业银行(北京怀北庄支行),北京市,NaN,NaN,62425a06a095bf8413293555
8,北京市怀柔区雁栖经济开发区雁栖路31号,怀柔区,北京市,1,40.358444,116.690489,中国农业银行(北京雁栖开发区支行),北京市,9e8cea88d259c725c51b75ca,(010)61665856,9e8cea88d259c725c51b75ca
9,北京市怀柔区大星发农发地农副产品批发市场,怀柔区,北京市,1,40.342267,116.652559,中国农业银行ATM,北京市,NaN,NaN,15bd45f5c7ae5b6cf603b807


In [117]:
results[results['area']=='海淀区']

,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
1,北京市海淀区清河镇龙岗路20号,海淀区,北京市,1,40.036410,116.362553,中国农业银行(北京龙岗路支行),北京市,f448d513e6c91b620d3dda4b,"(010)62991550,(010)62996623",f448d513e6c91b620d3dda4b
2,北京市海淀区花园路甲2号院4号楼一层101,海淀区,北京市,1,39.986153,116.372868,中国农业银行(北京花园路支行),北京市,a2b093525238353000ec0120,(010)82247102,a2b093525238353000ec0120
8,海淀区蓝靛厂中路晨月园甲4号,海淀区,北京市,1,39.964899,116.290373,中国农业银行(北京世纪城支行),北京市,606289fd39b75f4ca304648f,NaN,606289fd39b75f4ca304648f
12,北京市海淀区北蜂窝路5号院2号楼,海淀区,北京市,1,39.910947,116.334905,中国农业银行(北京铁道支行),北京市,befdb292e60ee99c8715413b,NaN,befdb292e60ee99c8715413b
41,北京市海淀区西郊板井曙光花园中路11号北京农科大厦首层B103号,海淀区,北京市,1,39.949037,116.293928,中国农业银行(北京曙光支行),北京市,4642844a215104d9c9fa5f41,NaN,4642844a215104d9c9fa5f41
42,北京市海淀区西海国际中心1层,海淀区,北京市,1,39.969957,116.315115,中国农业银行(北京西海支行),北京市,548aa32bae1e6276c42ba11b,(010)82194280,548aa32bae1e6276c42ba11b
66,北京市海淀区清河小营西路,海淀区,北京市,1,40.042356,116.341820,中国农业银行(北京清河支行),北京市,0752d427705939723a13fd9c,(010)62911826,0752d427705939723a13fd9c
67,北京市海淀区马连洼北路东居物业写字楼一层,海淀区,北京市,1,40.039892,116.293431,中国农业银行(北京竹园支行),北京市,25ec69fc59ab547dc26c146c,(010)82899445,25ec69fc59ab547dc26c146c
68,北京市海淀区上地三街9号嘉华大厦a座1楼,海淀区,北京市,1,40.042110,116.313938,中国农业银行(北京上地支行),北京市,91c13f93bba2cffa21f0176c,(010)62967285,91c13f93bba2cffa21f0176c
91,北京市海淀区复兴路28号中国人民解放军总医院门诊楼一层东侧2区房,海淀区,北京市,1,39.912694,116.283432,中国农业银行(五棵松储蓄所),北京市,a892d50d4830149f6f3e1b8a,NaN,a892d50d4830149f6f3e1b8a


In [72]:
nearby_venues = json_normalize(venues) # flatten JSON

In [73]:
#nearby_venues

In [74]:
# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
print(nearby_venues.shape)

nearby_venues

(20, 11)


,address,area,city,detail,lat,lng,name,province,street_id,telephone,uid
0,北京市朝阳区望京商业街首层A01,朝阳区,北京市,1,40.014711,116.473317,中国农业银行(北京望京支行),北京市,5eca57d48fd9286a9e9d685c,(010)64711745,5eca57d48fd9286a9e9d685c
1,北京市朝阳区杨闸环岛朗廷大厦1层,朝阳区,北京市,1,39.917008,116.605053,中国农业银行(北京八里桥支行),北京市,53cebd504a6afc602a02e162,NaN,53cebd504a6afc602a02e162
2,北京市朝阳区新源南路13号琨莎中心东侧,朝阳区,北京市,1,39.953933,116.459226,中国农业银行(北京新源里支行),北京市,e6ee2fd2d682fc362a11d1b6,(010)64675046,e6ee2fd2d682fc362a11d1b6
3,北京市朝阳区太阳宫南街13号楼1层,朝阳区,北京市,1,39.980271,116.450977,中国农业银行(北京太阳宫支行),北京市,12741f2408eafe8df4b9163b,(010)84150969,12741f2408eafe8df4b9163b
4,花家地街1号,朝阳区,北京市,1,39.991181,116.476041,中国农业银行(北京将台路支行),北京市,cc788fde05982d8a88fc064c,(010)64722757,cc788fde05982d8a88fc064c
5,北京市朝阳区麦子店街78号,朝阳区,北京市,1,39.950969,116.478695,中国农业银行(北京康运支行),北京市,78bd50f75112585b31870a4c,"95599,(010)65042614",78bd50f75112585b31870a4c
6,北京市朝阳区霄云路32号,朝阳区,北京市,1,39.963700,116.470767,中国农业银行(北京三元支行),北京市,3a0a78bdd1a317fa6318094c,(010)64624386,3a0a78bdd1a317fa6318094c
7,北京市朝阳区朝阳北路219号天伦伟业大厦一层及二层,朝阳区,北京市,1,39.929329,116.472553,中国农业银行(北京水碓西里支行),北京市,e3e2cd9d8089bbe3c7f50d4c,(010)85983473,e3e2cd9d8089bbe3c7f50d4c
8,北京市朝阳区红军营南路辅路,朝阳区,北京市,1,40.039250,116.422294,中国农业银行(北京北苑支行),北京市,NaN,NaN,55290d03c0947d03581093aa
9,北京市朝阳区宏昌路望京西园二区221号,朝阳区,北京市,1,40.007981,116.479234,中国农业银行(北京望盛园支行),北京市,4edf9fbff0991394cc06a47a,(010)64789601,4edf9fbff0991394cc06a47a
